# Required libraries

In [ ]:
!huggingface-cli login

In [ ]:
!pip install -q transformers accelerate bitsandbytes gradio

In [ ]:
!pip install peft

In [ ]:
!pip install langchain faiss-cpu sentence-transformers

In [ ]:
!pip install -U langchain-community

#Rag Database

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.load_local(
    "/content/drive/MyDrive/rag_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)


# Data structure and Memory System

In [ ]:
import torch
import json
import os
from datetime import datetime # Import datetime
from typing import Dict, List
import gradio as gr

class SimpleDataManager:
    """Simple file-based storage for user data and recommendations"""

    def __init__(self, data_dir="wellbeing_data"):
        self.data_dir = data_dir
        os.makedirs(data_dir, exist_ok=True)
        os.makedirs(f"{data_dir}/users", exist_ok=True)
        os.makedirs(f"{self.data_dir}/recommendations", exist_ok=True) # Corrected path

    def save_user_week(self, user_id: str, week_data: Dict):
        """Save weekly data for a user"""
        user_file = f"{self.data_dir}/users/{user_id}.json"

        try:
            # Load existing data
            if os.path.exists(user_file):
                with open(user_file, 'r') as f:
                    user_data = json.load(f)
            else:
                user_data = {"user_id": user_id, "weeks": []}

            # Add new week
            user_data["weeks"].append(week_data)

            # Keep only last 8 weeks
            user_data["weeks"] = user_data["weeks"][-8:]

            # Save
            with open(user_file, 'w') as f:
                json.dump(user_data, f, indent=2)

            print(f"✅ Saved week data for user {user_id}")

        except Exception as e:
            print(f"❌ Error saving user data: {str(e)}")

    def get_user_history(self, user_id: str) -> List[Dict]:
        """Get user's weekly history"""
        user_file = f"{self.data_dir}/users/{user_id}.json"

        try:
            if os.path.exists(user_file):
                with open(user_file, 'r') as f:
                    user_data = json.load(f)
                    history = user_data.get("weeks", [])
                    print(f"📊 Retrieved {len(history)} weeks of history for {user_id}")
                    return history
        except Exception as e:
            print(f"❌ Error loading user history: {str(e)}")

        return []

    def save_recommendation(self, user_id: str, week_start: str, recommendation: str):
        """Save LLM recommendation"""
        # Clean week_start for filename (remove invalid characters)
        clean_week = week_start.replace("/", "-").replace(":", "-")
        rec_file = f"{self.data_dir}/recommendations/{user_id}_{clean_week}.json"

        try:
            rec_data = {
                "user_id": user_id,
                "week_start": week_start,
                "recommendation": recommendation,
                "timestamp": datetime.now().isoformat()
            }

            with open(rec_file, 'w') as f:
                json.dump(rec_data, f, indent=2)

            print(f"✅ Saved recommendation for user {user_id}, week {week_start}")

        except Exception as e:
            print(f"❌ Error saving recommendation: {str(e)}")

    def get_last_recommendation(self, user_id: str) -> str:
        """Get user's last recommendation"""
        rec_dir = f"{self.data_dir}/recommendations"

        try:
            # Check if recommendations directory exists
            if not os.path.exists(rec_dir):
                return ""

            # Find latest recommendation file for this user
            user_files = [f for f in os.listdir(rec_dir) if f.startswith(f"{user_id}_") and f.endswith('.json')]

            if user_files:
                # Sort by date and get latest
                user_files.sort(reverse=True)
                latest_file = f"{rec_dir}/{user_files[0]}"

                with open(latest_file, 'r') as f:
                    rec_data = json.load(f)
                    recommendation = rec_data.get("recommendation", "")
                    print(f"📝 Retrieved last recommendation for {user_id} ({len(recommendation)} chars)")
                    return recommendation

        except Exception as e:
            print(f"❌ Error loading last recommendation: {str(e)}")

        return ""

    def get_user_stats(self, user_id: str) -> Dict:
        """Get basic stats about user's data (bonus method)"""
        history = self.get_user_history(user_id)

        if not history:
            return {"total_weeks": 0}

        return {
            "total_weeks": len(history),
            "first_week": history[0].get("week_start", "Unknown"),
            "latest_week": history[-1].get("week_start", "Unknown"),
            "avg_steps": sum(week.get("total_steps", 0) for week in history) // len(history),
            "avg_sleep": sum(week.get("avg_sleep", 0) for week in history) / len(history)
        }

#Well being LLM

In [ ]:
import torch
import re
from typing import Dict, List


class WellbeingLLM:
    """Improved LLM system for wellbeing recommendations with better prompt engineering"""

    def __init__(self, base_model_id: str, adapter_path: str, vectorstore_path: str = None):
        self.model = None
        self.tokenizer = None
        self.device = None
        self.vectorstore = None

        # Load vectorstore if path provided
        if vectorstore_path:
            self.load_vectorstore(vectorstore_path)

        self.load_model(base_model_id, adapter_path)

    def load_vectorstore(self, vectorstore_path: str):
        """Load FAISS vectorstore for RAG"""
        try:
            from langchain.vectorstores import FAISS
            from langchain.embeddings import HuggingFaceEmbeddings

            print("📚 Loading knowledge base...")
            embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

            self.vectorstore = FAISS.load_local(
                vectorstore_path,
                embeddings=embedding_model,
                allow_dangerous_deserialization=True
            )
            print("✅ Knowledge base loaded")

        except Exception as e:
            print(f"⚠️ Could not load vectorstore: {e}")
            self.vectorstore = None

    def load_model(self, base_model_id: str, adapter_path: str = None):
        """Load the LLM model with better error handling"""
        try:
            print("🤖 Loading LLM...")

            from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
            from peft import PeftModel

            # Clear GPU memory and check device
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                print(f"🔍 GPU available: {torch.cuda.get_device_name()}")
            else:
                print("🔍 Using CPU")

            # Load tokenizer
            print("📝 Loading tokenizer...")
            self.tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            print("✅ Tokenizer loaded")

            # Quantization config
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=torch.bfloat16,
            )
            print("⚙️ Quantization config ready")

            # Load base model
            print("🧠 Loading base model...")
            self.model = AutoModelForCausalLM.from_pretrained(
                base_model_id,
                device_map="auto",
                torch_dtype=torch.float16,  # Faster than float16
                quantization_config=bnb_config,
                trust_remote_code=True,
                low_cpu_mem_usage=True,      # 🚀 NEW: Reduces CPU memory usage
                use_cache=False,             # 🚀 NEW: Faster loading
            )
            print("✅ Base model loaded")

            # Load fine-tuned adapter
            # print("🎯 Loading fine-tuned adapter...")
            # self.model = PeftModel.from_pretrained(self.model, adapter_path)
            self.model.eval()

            # Store device for later use
            self.device = next(self.model.parameters()).device
            print(f"✅ LLM loaded successfully on {self.device}")

        except Exception as e:
            print(f"❌ LLM loading failed: {str(e)}")
            import traceback
            traceback.print_exc()
            self.model = None
            self.tokenizer = None
            self.device = None






    def generate_recommendation(self, current_week: Dict, user_history: List[Dict], last_recommendation: str) -> str:
      """Generate personalized recommendations with single template enforcement"""

      if not self.model or not self.tokenizer:
          print("❌ No model loaded")
          return "Model not available. Please check model loading."

      try:



          # Use the single master template
          query = self._build_master_template(current_week, user_history)


          docs = self.vectorstore.similarity_search(query, k=3)


          context = "\n\n".join(doc.page_content for doc in docs)
          print(f"the given prompt is {query}")


          prompt = (
          f"You are a helpful medical assistant. Use the following context to answer the question.\n\n"
          f"Context:\n{context}\n\n"
          f"Question: {query}\n\nAnswer:"
      )
          inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
          input_len = inputs["input_ids"].shape[1]

          print(f"Model is on: {self.device}")  # Should be 'cuda'


          with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=500,
                temperature=0.1,
                top_p=0.95,
                do_sample=True
            )

          generated_tokens = outputs[0][input_len:]
          answer = self.tokenizer.decode(generated_tokens, skip_special_tokens=True)

          return answer.strip()

      except Exception as e:
          print(f"❌ Generation error: {e}")
          return "Failed to generate response."


    def _build_master_template(self, current_week: Dict, user_history: List[Dict]) -> str:
      """Single master template with RAG-enhanced knowledge"""

      # Extract user data
      total_steps = current_week.get('total_steps', 0)
      zone_minutes = current_week.get('zone_minutes', 0)
      demographics = current_week.get('demographics', {})
      age = demographics.get('age', 'unknown')
      weight = demographics.get('weight', 'unknown')
      height = demographics.get('height', 'unknown')
      sex = demographics.get('sex', 'unknown')
      avg_sleep = current_week.get('avg_sleep', 0)
      diet_type = current_week.get('preferences', {}).get('diet_type', 'No Preference')
      allergies = current_week.get('preferences', {}).get('allergies', [])
      exercise_sessions = current_week.get('exercise_sessions', 0)
      food_data = current_week.get('food_data', {})
      dairy_liters = food_data.get('dairy_liters',0)
      legumes = food_data.get('legumes_grams',0)
      meat = food_data.get('meat_grams',0)
      furits= food_data.get('fruits_grams',0)
      vegtables = food_data.get('vegetables_grams',0)
      grains = food_data.get('grains_grams',0)
      nuts = food_data.get('nuts_seeds_grams',0)
      water = food_data.get('water_liters',0)



      # The master template with RAG integration
      prompt = f"""
A user is a {sex} aged {age} years, with a height of {height} cm and weight of {weight} kg. They walk about {total_steps:,} steps weekly, average {zone_minutes} zone minutes, and do {exercise_sessions} exercise sessions per week. They sleep around {avg_sleep:.1f} hours each night. Their diet preference is {diet_type}, and they are allergic to {', '.join(allergies) if allergies else 'none'}
and their this weekly food consumption is dairy {dairy_liters} liter, legumes {legumes} grams, meat {meat} grams, fruits {furits} grams, vegtables = {vegtables},
 grains {grains} grams, nuts {nuts} grams and water {water} liter .
---
**Important Dietary Notes:**
- Provide food suggestions appropriate for the user's diet preference ({diet_type}).
- If the user consumes foods that contradict their diet or allergies, highlight these inconsistencies and offer recommendations to align their consumption with their dietary needs.
---
Based on this profile and the expert knowledge below, provide health and wellness recommendations in the format outlined.

---

**1) Food Recommendation**
- Overall Assessment: [...]
- Areas of Improvements: [...]
- Suggested Meals: [...]

**2) Physical Activity**
- Activity Assessment: [...]
- Zone Minutes and intensity Feedback
- Strength/Cardio Tips: [suggest also exercise for strength and cardio]
- Weekly Goals: [...]

**3) Sleep & Well-being**
- Sleep Review: [...]
- Suggestions: [...]

**4) Weekly Summary**
- Summary: [...]
- Goals: [...]
"""


      return prompt




# Well being system

In [ ]:
class WellbeingSystem:
    """Main system that combines data management and LLM"""

    def __init__(self, base_model_id: str, adapter_path: str, vectorstore_path: str = None):
        self.data_manager = SimpleDataManager()
        self.llm = WellbeingLLM(base_model_id, adapter_path, vectorstore_path) # Pass vectorstore_path to LLM
        print("🎯 WellbeingSystem initialized")

    def analyze_and_recommend(self, user_id: str, week_data: Dict) -> str:
        """Main function: analyze weekly data and generate recommendations"""

        print(f"📊 Analyzing data for user: {user_id}")

        try:
            # Get user history and last recommendation
            user_history = self.data_manager.get_user_history(user_id)
            last_recommendation = self.data_manager.get_last_recommendation(user_id)

            print(f"📈 Found {len(user_history)} weeks of history")
            if last_recommendation:
                print(f"📝 Previous recommendation found ({len(last_recommendation)} chars)")

            # Process current week data
            processed_week = self._process_week_data(week_data)

            # Show what data is being sent to LLM
            print(f"📊 Processed data: {processed_week['total_steps']:,} steps, {processed_week['zone_minutes']} zone mins, {processed_week['avg_sleep']:.1f}h sleep")

            # Generate recommendation using LLM
            recommendation = self.llm.generate_recommendation(
                processed_week, user_history, last_recommendation
            )

            # Save data (with error handling)
            try:
                self.data_manager.save_user_week(user_id, processed_week)
                self.data_manager.save_recommendation(user_id, processed_week['week_start'], recommendation)
                print(f"💾 Data saved successfully")
            except Exception as save_error:
                print(f"⚠️ Warning: Could not save data: {save_error}")
                # Continue anyway - return the recommendation even if saving fails

            print(f"✅ Generated recommendation for {user_id} ({len(recommendation)} chars)")
            return recommendation

        except Exception as e:
            print(f"❌ Error in analyze_and_recommend: {str(e)}")
            import traceback
            traceback.print_exc()
            return f"Analysis failed: {str(e)}. Please check your input data."

    def _process_week_data(self, week_data: Dict) -> Dict:
        """Process and standardize weekly data with better validation"""

        try:
            # Parse sleep and mood data with validation
            sleep_hours = week_data.get('sleep_hours', [])
            mood_scores = week_data.get('mood_scores', [])

            # Handle string input (comma-separated values)
            if isinstance(sleep_hours, str):
                try:
                    sleep_hours = [float(x.strip()) for x in sleep_hours.split(',') if x.strip()]
                except ValueError:
                    print("⚠️ Invalid sleep hours format, using default")
                    sleep_hours = []

            if isinstance(mood_scores, str):
                try:
                    mood_scores = [float(x.strip()) for x in mood_scores.split(',') if x.strip()]
                except ValueError:
                    print("⚠️ Invalid mood scores format, using default")
                    mood_scores = []

            # Calculate averages with validation
            avg_sleep = sum(sleep_hours) / len(sleep_hours) if sleep_hours else 0
            avg_mood = sum(mood_scores) / len(mood_scores) if mood_scores else 0

            # 🚀 ADD DEBUG:
            print(f"🔍 DEBUG - Sleep calculation: {sleep_hours} → avg: {avg_sleep}")
            print(f"🔍 DEBUG - Mood calculation: {mood_scores} → avg: {avg_mood}")

            # Enhanced food data processing
            food_data = self._process_food_data(week_data)

            processed_data = {
                'week_start': week_data.get('week_start', datetime.now().strftime('%Y-%m-%d')),
                'total_steps': max(0, int(week_data.get('total_steps', 0))), # Ensure int and non-negative
                'zone_minutes': max(0, int(week_data.get('zone_minutes', 0))), # Ensure int and non-negative
                'exercise_sessions': max(0, int(week_data.get('exercise_sessions', 0))), # Ensure int and non-negative
                'avg_sleep': round(avg_sleep, 1),
                'avg_mood': round(avg_mood, 1),
                'food_data': food_data,
                'timestamp': datetime.now().isoformat(),
                'demographics': week_data.get('demographics', {})
            }

            # Include preferences and targets if present in the input week_data
            if 'preferences' in week_data:
                processed_data['preferences'] = week_data['preferences']
            if 'targets' in week_data:
                processed_data['targets'] = week_data['targets']


            return processed_data

        except Exception as e:
            print(f"❌ Error processing week data: {str(e)}")
            # Return empty or default data structure on error
            return {
                 'week_start': week_data.get('week_start', datetime.now().strftime('%Y-%m-%d')),
                 'total_steps': 0,
                 'zone_minutes': 0,
                 'exercise_sessions': 0,
                 'avg_sleep': 0.0,
                 'avg_mood': 0.0,
                 'food_data': {},
                 'timestamp': datetime.now().isoformat(),
                 'processing_error': str(e)
            }


    def _process_food_data(self, week_data: Dict) -> Dict:
        """Process food data handling both old and new formats"""

        food_data = {}

        # Check for new format first (with units)
        unit_fields = ['dairy_liters', 'water_liters', 'legumes_grams', 'meat_grams',
                      'fruits_grams', 'vegetables_grams', 'grains_grams', 'nuts_seeds_grams']

        has_unit_format = any(field in week_data for field in unit_fields)

        if has_unit_format:
            print("📊 Using new format (with units)")
            # New format with units
            for field in unit_fields:
                value = week_data.get(field, 0)
                try:
                    food_data[field] = max(0, float(value))  # Ensure non-negative
                except (ValueError, TypeError):
                    food_data[field] = 0
                    print(f"⚠️ Invalid {field} value, using 0")
        else:
            print("📊 Using old format (servings)")
            # Old format (for backward compatibility)
            old_fields = ['dairy', 'legumes', 'meat', 'fruits', 'vegetables', 'grains', 'nuts_seeds', 'water_glasses']
            for field in old_fields:
                if field in week_data:
                    try:
                        food_data[field] = max(0, float(week_data[field]))
                    except (ValueError, TypeError):
                        food_data[field] = 0

        return food_data





# Well being app

In [ ]:
import gradio as gr
import json
import os
from datetime import datetime
from typing import Dict, List

class WellbeingApp:
    """Complete Gradio interface with targets, history, and preferences"""

    def __init__(self, base_model_id: str, adapter_path: str):
      print("🚀 Initializing Complete WellbeingApp...")
      self.wellbeing_system = WellbeingSystem(
          base_model_id,
          adapter_path,
          vectorstore_path="/content/drive/MyDrive/rag_index"
      )

      # Default targets (same as before)
      self.default_targets = {
          'weekly_steps': 70000,
          'weekly_zone_minutes': 150,
          'daily_sleep_hours': 8.0,
          'weekly_water_liters': 14.0,
          'weekly_fruits_grams': 2100,
          'weekly_vegetables_grams': 3500,
          'weekly_exercise_sessions': 4
      }

    def create_wellbeing_app(self):
        """Create comprehensive Gradio interface with tabs"""

        with gr.Blocks(title="🏥 Complete Wellbeing LLM", theme=gr.themes.Soft()) as demo:

            gr.Markdown("# 🏥 Complete AI Wellbeing System")
            gr.Markdown("### Personalized health recommendations with custom targets and progress tracking")

            # Create tabs for different sections
            with gr.Tabs():

                # TAB 1: MAIN ANALYSIS
                with gr.TabItem("🤖 Get Recommendations", elem_id="main-tab"):
                    self._create_main_analysis_tab()

                # TAB 2: TARGETS CUSTOMIZATION
                with gr.TabItem("🎯 Customize Targets", elem_id="targets-tab"):
                    self._create_targets_tab()

                # TAB 3: HISTORY REVIEW
                with gr.TabItem("📈 Progress History", elem_id="history-tab"):
                    self._create_history_tab()

                # TAB 4: SYSTEM INFO
                # with gr.TabItem("ℹ️ System Info", elem_id="info-tab"):
                #     self._create_info_tab()

        return demo

    def _create_main_analysis_tab(self):
      """Create the main analysis tab with fixed component handling"""

      with gr.Row():
          with gr.Column():

              gr.Markdown("### 🤖 AI Enhancement Options")
              # User Info Section
              gr.Markdown("### 👤 User Information")
              user_id = gr.Textbox(label="👤 User ID", value="user_001", info="Unique identifier for tracking your progress")
              week_start = gr.Textbox(label="📅 Week Start Date", value="2024-01-15", info="Format: YYYY-MM-DD")

              # Personal Details - FIXED: These were not being collected properly
              sex = gr.Dropdown(
                  label="⚧️ Sex",
                  choices=["Male", "Female", "Other", "Prefer not to say"],
                  value="Male",
                  info="Biological sex for health calculations"
              )

              age = gr.Number(
                  label="🎂 Age",
                  value=25,
                  minimum=13,
                  maximum=120,
                  step=1,
                  info="Age in years"
              )

              weight = gr.Number(
                  label="⚖️ Weight (kg)",
                  value=70.0,
                  minimum=30.0,
                  maximum=300.0,
                  step=0.1,
                  info="Current weight in kilograms"
              )

              height = gr.Number(
                  label="📏 Height (cm)",
                  value=175.0,
                  minimum=100.0,
                  maximum=250.0,
                  step=0.1,
                  info="Height in centimeters"
              )

              # Preferences Section
              gr.Markdown("### 🍽️ Meal Preferences & Dietary Restrictions")
              with gr.Row():
                  diet_type = gr.Dropdown(
                      label="🥗 Diet Type",
                      choices=[
                          "No Preference", "Vegetarian", "Vegan", "Pescatarian",
                          "Keto", "Paleo", "Mediterranean", "Low Carb", "Gluten Free", "Dairy Free"
                      ],
                      value="No Preference",
                      info="Select your primary dietary preference"
                  )

              allergies = gr.CheckboxGroup(
                  label="⚠️ Food Allergies & Intolerances",
                  choices=[
                      "Nuts (Tree nuts)", "Peanuts", "Shellfish", "Fish",
                      "Milk/Dairy", "Eggs", "Soy", "Wheat/Gluten",
                      "Sesame", "Lactose Intolerant", "Other"
                  ],
                  info="⚠️ IMPORTANT: Select all allergies and intolerances"
              )

              other_allergies = gr.Textbox(
                  label="🚨 Other Allergies/Restrictions",
                  placeholder="e.g., tomatoes, specific medications, religious restrictions...",
                  info="Specify any other dietary restrictions or allergies"
              )

              # Activity Section
              gr.Markdown("### 🏃‍♂️ Physical Activity")
              total_steps = gr.Number(label="Total Steps (Week)", value=65000, info="Your current target will be shown")
              zone_minutes = gr.Number(label="Zone Minutes (Week)", value=120, info="Your current target will be shown")
              exercise_sessions = gr.Number(label="Exercise Sessions", value=3, info="Your current target will be shown")

              # Sleep & Mood Section
              gr.Markdown("### 😴 Sleep & Mood")
              sleep_hours = gr.Textbox(
                  label="Sleep Hours (7 days)",
                  value="7.5,6.8,8.2,7.0,6.5,8.5,7.8",
                  info="Enter daily sleep hours separated by commas"
              )
              mood_scores = gr.Textbox(
                  label="Mood Scores (7 days)",
                  value="4,3.5,5,3,4,2.5,5",
                  info="Rate your daily mood from 1-5, separated by commas"
              )

              # Food Section
              gr.Markdown("### 🍎 Weekly Nutrition Intake")
              gr.Markdown("*Enter your total consumption for the entire week*")

              with gr.Row():
                  dairy = gr.Number(label="🥛 Dairy (liters)", value=3.5, info="Milk, yogurt, cheese")
                  water_liters = gr.Number(label="💧 Water (liters)", value=48, info="Your current target will be shown")

              with gr.Row():
                  fruits = gr.Number(label="🍎 Fruits (grams)", value=2100, info="Your current target will be shown")
                  vegetables = gr.Number(label="🥬 Vegetables (grams)", value=3500, info="Your current target will be shown")

              with gr.Row():
                  legumes = gr.Number(label="🫘 Legumes (grams)", value=350, info="Beans, lentils, peas")
                  nuts_seeds = gr.Number(label="🥜 Nuts/Seeds (grams)", value=210, info="Almonds, walnuts, chia seeds")

              with gr.Row():
                  meat = gr.Number(label="🍗 Meat (grams)", value=700, info="Chicken, beef, fish")
                  grains = gr.Number(label="🌾 Grains (grams)", value=2800, info="Rice, bread, pasta, oats")

              # Action buttons
              with gr.Row():
                  analyze_btn = gr.Button("🤖 Get AI Recommendations", variant="primary", size="lg")
                  load_targets_btn = gr.Button("📊 Load My Targets", variant="secondary")

          with gr.Column():
              main_output = gr.Markdown("""
  Suggestions
  """)

      # FIXED: Store components with correct order including demographics
      self.main_components = {
          'user_id': user_id,
          'week_start': week_start,
          'sex': sex,           # ADDED: Missing demographic fields
          'age': age,           # ADDED
          'weight': weight,     # ADDED
          'height': height,     # ADDED
          'total_steps': total_steps,
          'zone_minutes': zone_minutes,
          'exercise_sessions': exercise_sessions,
          'sleep_hours': sleep_hours,
          'mood_scores': mood_scores,
          'dairy_liters': dairy,
          'legumes_grams': legumes,
          'meat_grams': meat,
          'fruits_grams': fruits,
          'vegetables_grams': vegetables,
          'grains_grams': grains,
          'nuts_seeds_grams': nuts_seeds,
          'water_liters': water_liters,
          'diet_type': diet_type,
          'allergies': allergies,
          'other_allergies': other_allergies,
          'output': main_output
      }

      # FIXED: Connect button with correct inputs order
      analyze_btn.click(
          fn=self.analyze_wellbeing_with_preferences,
          inputs=[
              user_id, week_start, sex, age, weight, height,  # Demographics in correct order
              total_steps, zone_minutes, exercise_sessions,
              sleep_hours, mood_scores, dairy, legumes, meat,
              fruits, vegetables, grains, nuts_seeds, water_liters,
              diet_type, allergies, other_allergies
          ],
          outputs=[main_output]
      )

      load_targets_btn.click(
          fn=self.load_user_targets,
          inputs=[user_id],
          outputs=[main_output]
      )

    def analyze_wellbeing_with_preferences(self, user_id, week_start, sex, age, weight, height,
                                     total_steps, zone_minutes, exercise_sessions,
                                     sleep_hours, mood_scores, dairy_liters, legumes_grams, meat_grams,
                                     fruits_grams, vegetables_grams, grains_grams, nuts_seeds_grams, water_liters,
                                     diet_type, allergies, other_allergies):
      """Enhanced analysis with agent support, custom targets, and user demographics"""

      try:
          # Input validation
          if not user_id or not user_id.strip():
              return "❌ Error: Please enter a valid User ID"

          if not week_start:
              return "❌ Error: Please enter a week start date"

          # Validate demographic data
          if age is not None and (age < 13 or age > 120):
              return "❌ Error: Please enter a valid age (13-120 years)"

          if weight is not None and (weight < 30 or weight > 300):
              return "❌ Error: Please enter a valid weight (30-300 kg)"

          if height is not None and (height < 100 or height > 250):
              return "❌ Error: Please enter a valid height (100-250 cm)"

          # Load user's custom targets
          user_targets = self.get_user_targets(user_id.strip())

          # Calculate BMI if weight and height are provided
          bmi = None
          if weight and height:
              height_m = height / 100  # Convert cm to meters
              bmi = round(weight / (height_m ** 2), 1)

          # Prepare enhanced data with demographics
          week_data = {
              'user_id': user_id.strip(),
              'week_start': week_start,
              'demographics': {
                  'sex': sex,
                  'age': int(age) if age else None,
                  'weight': float(weight) if weight else None,
                  'height': float(height) if height else None,
                  'bmi': bmi
              },
              'total_steps': max(0, int(total_steps)) if total_steps else 0,
              'zone_minutes': max(0, int(zone_minutes)) if zone_minutes else 0,
              'exercise_sessions': max(0, int(exercise_sessions)) if exercise_sessions else 0,
              'sleep_hours': sleep_hours,
              'mood_scores': mood_scores,
              'dairy_liters': max(0, float(dairy_liters)) if dairy_liters else 0,
              'legumes_grams': max(0, float(legumes_grams)) if legumes_grams else 0,
              'meat_grams': max(0, float(meat_grams)) if meat_grams else 0,
              'fruits_grams': max(0, float(fruits_grams)) if fruits_grams else 0,
              'vegetables_grams': max(0, float(vegetables_grams)) if vegetables_grams else 0,
              'grains_grams': max(0, float(grains_grams)) if grains_grams else 0,
              'nuts_seeds_grams': max(0, float(nuts_seeds_grams)) if nuts_seeds_grams else 0,
              'water_liters': max(0, float(water_liters)) if water_liters else 0,
              'preferences': {
                  'diet_type': diet_type,
                  'allergies': allergies if allergies else [],
                  'other_allergies': other_allergies.strip() if other_allergies else ""
              },
              'targets': user_targets
          }

          recommendation = self.wellbeing_system.analyze_and_recommend(user_id.strip(), week_data)

          # Enhanced output with custom targets and demographics
          output = self._format_analysis_output_with_targets(week_data, recommendation, user_targets)

          return output

      except Exception as e:
          print(f"❌ Analysis error: {str(e)}")
          import traceback
          traceback.print_exc()
          return f"❌ Error: {str(e)}\n\nPlease check your input format and try again."


    def _create_targets_tab(self):
        """Create the targets customization tab"""

        gr.Markdown("## 🎯 Customize Your Personal Health Targets")
        gr.Markdown("### Set personalized goals based on your fitness level, health conditions, and personal objectives")

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 👤 User Selection")
                target_user_id = gr.Textbox(label="👤 User ID", value="user_001", info="Enter the user ID to customize targets for")

                gr.Markdown("### 🏃‍♂️ Activity Targets")
                target_weekly_steps = gr.Number(
                    label="Weekly Steps Target",
                    value=70000,
                    info="Default: 70,000 (10,000 daily). Adjust based on fitness level."
                )
                target_zone_minutes = gr.Number(
                    label="Weekly Zone Minutes Target",
                    value=150,
                    info="Default: 150 minutes. WHO recommends 150+ minutes moderate exercise."
                )
                target_exercise_sessions = gr.Number(
                    label="Weekly Exercise Sessions Target",
                    value=4,
                    info="Default: 4 sessions. Include both cardio and strength training."
                )

                gr.Markdown("### 😴 Sleep & Recovery Targets")
                target_daily_sleep = gr.Number(
                    label="Daily Sleep Hours Target",
                    value=8.0,
                    info="Default: 8 hours. Range: 7-9 hours for most adults."
                )

                with gr.Row():
                    save_targets_btn = gr.Button("💾 Save My Targets", variant="primary", size="lg")
                    load_current_targets_btn = gr.Button("📊 Load Current Targets", variant="secondary")
                    reset_defaults_btn = gr.Button("🔄 Reset to Defaults", variant="secondary")


        # Store target components
        self.target_components = {
            'user_id': target_user_id,
            'weekly_steps': target_weekly_steps,
            'zone_minutes': target_zone_minutes,
            'exercise_sessions': target_exercise_sessions,
            'daily_sleep': target_daily_sleep,
        }

        # Connect buttons
        save_targets_btn.click(
            fn=self.save_user_targets,
            inputs=list(self.target_components.values())[:-1],
            # outputs=[targets_output]
        )

        load_current_targets_btn.click(
            fn=self.load_user_targets_display,
            inputs=[target_user_id],
            outputs=list(self.target_components.values()) # Update all target number boxes and output
        )

        reset_defaults_btn.click(
            fn=self.reset_to_defaults,
            outputs=list(self.target_components.values())
        )


    def _create_history_tab(self):
        """Create the history review tab"""

        gr.Markdown("## 📈 Progress History & Analytics")
        gr.Markdown("### Track your health journey and see improvements over time")

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 👤 User Selection")
                history_user_id = gr.Textbox(label="👤 User ID", value="user_001", info="Enter user ID to view history")

                gr.Markdown("### 📊 Analysis Options")
                analysis_type = gr.Radio(
                    label="📈 View Type",
                    choices=[
                        "📊 Complete Progress Summary",
                        "📈 Weekly Trends Analysis",
                        "🎯 Target Achievement Report",
                        "📋 Raw Data Export",
                        "🏆 Achievement Milestones"
                    ],
                    value="📊 Complete Progress Summary",
                    info="Choose what type of analysis you want to see"
                )

                weeks_to_show = gr.Slider(
                    label="📅 Weeks to Include",
                    minimum=1,
                    maximum=12,
                    value=4,
                    step=1,
                    info="How many recent weeks to analyze"
                )

                with gr.Row():
                    view_history_btn = gr.Button("📊 View Progress", variant="primary", size="lg")
                    export_data_btn = gr.Button("💾 Export Data", variant="secondary")
                    delete_history_btn = gr.Button("🗑️ Clear History", variant="stop")



        # Store history components
        self.history_components = {
            'user_id': history_user_id,
            'analysis_type': analysis_type,
            'weeks_to_show': weeks_to_show,
            # 'output': history_output
        }

    def _format_preferences_display(self, preferences: dict) -> str:
        """Format preferences for nice display"""

        lines = ["### 🍽️ Your Dietary Profile:"]

        # Diet type
        if preferences.get('diet_type', 'No Preference') != "No Preference":
            lines.append(f"- **Diet:** {preferences['diet_type']}")

        # Allergies (most important!)
        if preferences.get('allergies'):
            allergy_list = ", ".join(preferences['allergies'])
            lines.append(f"- **⚠️ ALLERGIES:** {allergy_list}")

        if preferences.get('other_allergies'):
            lines.append(f"- **⚠️ OTHER RESTRICTIONS:** {preferences['other_allergies']}")

        if len(lines) == 1:  # Only the header
            lines.append("- No specific dietary preferences set")

        return "\n".join(lines)



    def _format_analysis_output_with_targets(self, week_data, recommendation, user_targets):
      """Format analysis output including custom targets"""

      daily_steps = week_data['total_steps'] / 7
      daily_water = week_data['water_liters'] / 7

      # Safely get avg_sleep and avg_mood with defaults
      avg_sleep = week_data.get('avg_sleep', 0)
      avg_mood = week_data.get('avg_mood', 0)

      # Compare against custom targets
      targets_comparison = f"""
  ### 🎯 Target Performance:
  - **Steps:** {week_data['total_steps']:,} / {user_targets['weekly_steps']:,} {'✅' if week_data['total_steps'] >= user_targets['weekly_steps'] else '⚠️'} ({((week_data['total_steps'] / user_targets['weekly_steps']) * 100):.0f}%)
  - **Zone Minutes:** {week_data['zone_minutes']} / {user_targets['weekly_zone_minutes']} {'✅' if week_data['zone_minutes'] >= user_targets['weekly_zone_minutes'] else '⚠️'} ({((week_data['zone_minutes'] / user_targets['weekly_zone_minutes']) * 100):.0f}%)
  - **Sleep:** {avg_sleep:.1f}h / {user_targets['daily_sleep_hours']}h {'✅' if avg_sleep >= user_targets['daily_sleep_hours'] else '⚠️'}
  - **Water:** {week_data['water_liters']:.1f}L / {user_targets['weekly_water_liters']}L {'✅' if week_data['water_liters'] >= user_targets['weekly_water_liters'] else '⚠️'}
"""

      preferences_display = self._format_preferences_display(week_data['preferences'])

      return f"""# 🏥 Personalized Wellbeing Analysis & Recommendations

  ## 👤 User: {week_data['user_id']} | 📅 Week: {week_data['week_start']}

  {preferences_display}

  {targets_comparison}

  ### 📊 Your Weekly Data Summary:
  **🏃‍♂️ Activity:**
  - **Steps:** {week_data['total_steps']:,} ({daily_steps:.0f}/day)
  - **Zone Minutes:** {week_data['zone_minutes']}
  - **Exercise Sessions:** {week_data['exercise_sessions']}

  **😴 Sleep & Mood:**
  - **Sleep:** {week_data['sleep_hours']} hours daily
  - **Mood:** {week_data['mood_scores']}/10 daily scores

  **🍎 Nutrition:**
  - 🥛 **Dairy:** {week_data['dairy_liters']:.1f}L | 💧 **Water:** {week_data['water_liters']:.1f}L ({daily_water:.1f}L/day)
  - 🍎 **Fruits:** {week_data['fruits_grams']:.0f}g | 🥬 **Vegetables:** {week_data['vegetables_grams']:.0f}g
  - 🫘 **Legumes:** {week_data['legumes_grams']:.0f}g | 🍗 **Meat:** {week_data['meat_grams']:.0f}g
  - 🌾 **Grains:** {week_data['grains_grams']:.0f}g | 🥜 **Nuts/Seeds:** {week_data['nuts_seeds_grams']:.0f}g

  ---

  ## 🤖 AI Recommendations
  *Personalized based on your data, preferences, allergies, and custom targets\n*

  {recommendation}
  """

    def load_user_targets(self, user_id):
        """Load user targets and format for display in main tab"""
        try:
            if not user_id or not user_id.strip():
                return "❌ Please enter a valid User ID"

            targets = self.get_user_targets(user_id.strip())

            return f"""# 📊 Your Current Targets Loaded

## 🎯 Targets for {user_id}:

### 🏃‍♂️ Activity Targets:
- **Weekly Steps:** {targets['weekly_steps']:,}
- **Zone Minutes:** {targets['weekly_zone_minutes']}
- **Exercise Sessions:** {targets['weekly_exercise_sessions']}

### 😴 Recovery Target:
- **Daily Sleep:** {targets['daily_sleep_hours']} hours

### 🍎 Nutrition Targets:
- **Weekly Water:** {targets['weekly_water_liters']} liters
- **Weekly Fruits:** {targets['weekly_fruits_grams']:,} grams
- **Weekly Vegetables:** {targets['weekly_vegetables_grams']:,} grams

*These targets will be used to evaluate your progress when you get AI recommendations.*

**💡 Tip:** Go to the 'Customize Targets' tab to modify these values if needed.
"""

        except Exception as e:
            return f"❌ Error loading targets: {str(e)}"

    def save_user_targets(self, user_id, weekly_steps, zone_minutes, exercise_sessions,
                          daily_sleep, weekly_water):
        """Saves user's custom targets to a file"""
        if not user_id or not user_id.strip():
            return "❌ Error: Please enter a valid User ID to save targets."

        targets_file = f"{self.wellbeing_system.data_manager.data_dir}/targets_{user_id.strip()}.json"

        try:
            targets_data = {
                'weekly_steps': int(weekly_steps) if weekly_steps else self.default_targets['weekly_steps'],
                'weekly_zone_minutes': int(zone_minutes) if zone_minutes else self.default_targets['weekly_zone_minutes'],
                'weekly_exercise_sessions': int(exercise_sessions) if exercise_sessions else self.default_targets['weekly_exercise_sessions'],
                'daily_sleep_hours': float(daily_sleep) if daily_sleep else self.default_targets['daily_sleep_hours'],
                'weekly_water_liters': float(weekly_water) if weekly_water else self.default_targets['weekly_water_liters'],
              }

            with open(targets_file, 'w') as f:
                json.dump(targets_data, f, indent=2)

            return f"✅ Targets saved successfully for user '{user_id.strip()}'."

        except Exception as e:
            return f"❌ Error saving targets: {str(e)}"

    def get_user_targets(self, user_id: str) -> Dict:
        """Loads user's custom targets, or returns defaults"""
        targets_file = f"{self.wellbeing_system.data_manager.data_dir}/targets_{user_id.strip()}.json"

        if os.path.exists(targets_file):
            try:
                with open(targets_file, 'r') as f:
                    targets = json.load(f)
                    # Ensure all keys from default are present (for backward compatibility)
                    full_targets = self.default_targets.copy()
                    full_targets.update(targets)
                    return full_targets
            except Exception as e:
                print(f"⚠️ Warning: Could not load targets for user '{user_id.strip()}': {str(e)}. Using defaults.")
                return self.default_targets
        else:
            return self.default_targets

    def load_user_targets_display(self, user_id):
        """Loads user targets and formats for display in the targets tab"""
        if not user_id or not user_id.strip():
             return "❌ Error: Please enter a valid User ID to load targets.", *[gr.Number(value=self.default_targets[key]) for key in self.default_targets] # Return defaults to update UI

        targets = self.get_user_targets(user_id.strip())

        output_message = f"✅ Loaded targets for user '{user_id.strip()}'."

        # Return the message and the target values to update the Gradio Number components
        return (
            output_message,
            targets.get('weekly_steps', self.default_targets['weekly_steps']),
            targets.get('weekly_zone_minutes', self.default_targets['weekly_zone_minutes']),
            targets.get('weekly_exercise_sessions', self.default_targets['weekly_exercise_sessions']),
            targets.get('daily_sleep_hours', self.default_targets['daily_sleep_hours']),
            targets.get('weekly_water_liters', self.default_targets['weekly_water_liters']),
            )

    def reset_to_defaults(self):
        """Resets the target input fields to default values"""
        return (
            "🎯 Resetting targets to defaults.",
            self.default_targets['weekly_steps'],
            self.default_targets['weekly_zone_minutes'],
            self.default_targets['weekly_exercise_sessions'],
            self.default_targets['daily_sleep_hours'],
            self.default_targets['weekly_water_liters'],
        )









# Main execution

In [ ]:
# Uncomment to run:
app = WellbeingApp(base_model_id="ContactDoctor/Bio-Medical-Llama-3-8B", adapter_path="AnjaliNV/WellBeing_LLM")
demo = app.create_wellbeing_app()
demo.launch(share=True, debug=True)

# Fine Tuning

In [ ]:
import os
import json
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
import torch # Import torch to check for CUDA

# Load your dataset from JSONL

new_data = [
{"input": {"Demographics": {"Gender": "Male", "Age": 35, "Height": 180, "Weight": 85}, "Physical Activity": {"Weekly Steps": 6000, "Zone Minutes per week": 90, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1, "Legumes": 300, "Meat": 500, "Fruits": 400, "Vegetables": 500, "Grains": 1500, "Nuts": 100, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "Your diet is primarily Mediterranean, which is good, but there is a higher consumption of meat than ideal, and fruits and vegetables are on the lower side.", "Areas of Improvements": "Increase fruit intake to around 800g/week and vegetables to at least 700g/week.", "Suggested Meals": ["Breakfast: Greek yogurt topped with mixed berries and a sprinkle of nuts.", "Lunch: Quinoa salad with chickpeas, spinach, cherry tomatoes, and feta cheese, drizzled with olive oil.", "Dinner: Grilled salmon with a side of steamed broccoli and sweet potatoes."]}, "Physical Activity": {"Activity Assessment": "You are engaging in moderate physical activity, which is beneficial but could be improved.", "Zone Minutes and Intensity Feedback": "Aim to increase your zone minutes to around 150 per week for better cardiovascular health.", "Strength/Cardio Tips": "Incorporate a mix of strength training and cardio exercises, such as cycling or swimming, at least 2-3 times a week.", "Weekly Goals": "Try to increase your step count to at least 10,000 steps per day and add an additional exercise session."}, "Sleep & Well-being": {"Sleep Review": "Your average sleep of 6 hours per night is below the recommended amount for adults, which can affect overall well-being.", "Suggestions": "Aim for 7-8 hours of sleep by setting a regular sleep schedule and reducing screen time before bed."}, "Weekly Summary": {"Summary": "You have a mostly balanced Mediterranean diet, but with room for improvement in fruit and vegetable consumption and physical activity levels.", "Goals": "Increase fruit and vegetable intake, boost physical activity levels, and improve sleep duration."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 35, "Height": 178, "Weight": 85}, "Physical Activity": {"Weekly Steps": 7000, "Zone Minutes per week": 100, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "None"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 200, "Meat": 500, "Fruits": 300, "Vegetables": 600, "Grains": 400, "Nuts": 100, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The current diet shows a good variety of food groups, particularly vegetables and legumes, but there is room for improvement in fruit and nut intake.", "Areas of Improvements": "Increase fruit consumption from 300g to about 800g/week and nuts from 100g to 200g/week.", "Suggested Meals": "Breakfast: Greek yogurt with fresh berries and a sprinkle of nuts; Lunch: Quinoa salad with chickpeas, diced vegetables, and a side of mixed fruit; Dinner: Grilled fish with a side of steamed broccoli and roasted sweet potatoes."}, "Physical Activity": {"Activity Assessment": "Physical activity is moderate, with 2 exercise sessions per week and 7000 weekly steps, which is excellent for maintaining health but could be enhanced.", "Zone Minutes and Intensity Feedback": "100 zone minutes indicate some intensity; consider increasing to 150 for improved cardiovascular health.", "Strength/Cardio Tips": "Incorporate strength training at least once a week, such as bodyweight exercises or resistance bands, and aim for at least 30 minutes of higher intensity cardio per session.", "Weekly Goals": "Increase weekly steps to 10,000 and add an additional exercise session."}, "Sleep & Well-being": {"Sleep Review": "Average sleep of 6 hours is below the recommended 7-9 hours for optimal health and recovery.", "Suggestions": "Establish a consistent bedtime routine, limit screen time before bed, and consider relaxation techniques to improve sleep duration and quality."}, "Weekly Summary": {"Summary": "The profile indicates a fairly balanced diet with opportunities for improved fruit and nut intake, a moderate level of physical activity, and a need to enhance sleep quality.", "Goals": "Increase fruit and nut intake, reach 10,000 daily steps, add a third exercise session, and aim for 7-8 hours of sleep per night."}}},
{"input": {"Gender": "Male", "Age": 45, "Height": 178, "Weight": 85, "Physical Activity": {"Weekly Steps": 6000, "Zone Minutes": 90, "Exercise Sessions": 2}, "Sleep": {"Average hours": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 400, "Meat": 700, "Fruits": 300, "Vegetables": 500, "Grains": 1500, "Nuts": 200, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "Your current diet quality shows a good balance of legumes and grains, but fruits and vegetables could be more prominently featured.", "Areas of Improvements": "Consider increasing fruit intake to around 800g/week. Your vegetable intake is moderate, so aim to increase it to around 800g/week as well.", "Suggested Meals": ["Breakfast: Greek yogurt with mixed berries and a sprinkle of nuts.", "Lunch: Quinoa salad with chickpeas, cherry tomatoes, cucumber, and a drizzle of olive oil.", "Dinner: Grilled salmon with a side of roasted vegetables and a mixed green salad."]}, "Physical Activity": {"Activity Assessment": "Your current activity level is below recommended guidelines. Aim for at least 150 minutes of moderate-intensity aerobic activity each week.", "Zone Minutes and Intensity Feedback": "90 zone minutes indicates some effort, but try incorporating more brisk walks or jogs to reach higher intensity levels.", "Strength/Cardio Tips": "Incorporate strength training twice a week and consider adding a 30-minute brisk walk to your routine on non-exercise days.", "Weekly Goals": "Set a goal to increase weekly steps to at least 10,000 and accumulate at least 150 weekly zone minutes."}, "Sleep & Well-being": {"Sleep Review": "Getting an average of 6 hours of sleep may not suffice for optimal health. Adults generally need 7-9 hours.", "Suggestions": "Establish a calming bedtime routine, reduce screen time before bed, and try to go to sleep and wake up at the same time each day."}, "Weekly Summary": {"Summary": "Enhancing your fruit and vegetable intake, increasing physical activity, and improving sleep could significantly benefit your overall health and well-being.", "Goals": ["Increase fruit consumption to 800g/week.", "Increase vegetable consumption to 800g/week.", "Achieve at least 10,000 steps daily.", "Limit screen time before bed and aim for 7-8 hours of sleep."]}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height": 180, "Weight": 85}, "Physical Activity": {"Weekly Steps": 5000, "Zone Minutes per week": 80, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1, "Legumes": 300, "Meat": 400, "Fruits": 150, "Vegetables": 300, "Grains": 800, "Nuts": 100, "Water": 1.5}}, "output": {"Food Recommendation": {"Overall Assessment": "The user's diet follows a Mediterranean pattern but shows low fruit intake, which is crucial for vitamins and antioxidants.", "Areas of Improvements": "The user is currently consuming 150g/week of fruits; it's recommended to increase this to around 800g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with a mixed berry compote and a sprinkle of nuts.", "Lunch: Quinoa salad with chickpeas, cherry tomatoes, cucumber, and a drizzle of olive oil.", "Dinner: Grilled chicken with roasted vegetables and a side of fruit salad."]}, "Physical Activity": {"Activity Assessment": "Physical activity is below recommended levels for optimal health, with only 5,000 steps per week.", "Zone Minutes and Intensity Feedback": "The user engages in moderate activity with 80 zone minutes per week, which is adequate but could benefit from more high-intensity workouts.", "Strength/Cardio Tips": "Incorporate strength training exercises into the routine at least once a week, focusing on major muscle groups.", "Weekly Goals": "Aim for at least 8,000 steps per week and include one additional exercise session to boost fitness."}, "Sleep & Well-being": {"Sleep Review": "Average sleep duration of 6.5 hours is slightly below the recommended 7-9 hours for adults, which can impact overall health.", "Suggestions": "Establish a consistent bedtime routine, limit screen time before bed, and create a sleep-friendly environment to improve sleep quality."}, "Weekly Summary": {"Summary": "Overall, the user follows a fairly healthy Mediterranean diet but needs to increase fruit intake and physical activity levels. Improving sleep duration and quality is also essential.", "Goals": ["Increase fruit intake to 800g/week.", "Engage in physical activity to reach at least 8,000 steps weekly.", "Establish a regular sleep schedule aiming for 7-8 hours/night."]}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 45, "Height": 178, "Weight": 85}, "Physical Activity": {"Weekly Steps": 7000, "Zone Minutes": 80, "Exercise Sessions": 2}, "Sleep": {"Average Hours": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "No allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 300, "Meat": 600, "Fruits": 300, "Vegetables": 400, "Grains": 700, "Nuts": 150, "Water": 1.5}}, "output": {"Food Recommendation": {"Overall Assessment": "Your diet follows a Mediterranean pattern with a decent intake of legumes and vegetables, but you are consuming fewer fruits than recommended.", "Areas of Improvements": "Increase fruit consumption from 300g/week to around 800g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with mixed berries and a sprinkle of nuts.", "Lunch: Quinoa salad with chickpeas, diced cucumber, bell peppers, and a side of fresh fruit.", "Dinner: Grilled chicken with a side of sautéed spinach, cherry tomatoes, and a fruit salad for dessert."]}, "Physical Activity": {"Activity Assessment": "You're engaging in a moderate level of physical activity, but there's room for improvement in both weekly steps and exercise sessions.", "Zone Minutes and Intensity Feedback": "Your 80 zone minutes indicate some cardiovascular activity; try to increase this to at least 120 minutes for better health.", "Strength/Cardio Tips": "Incorporate at least one more session of strength training and add a weekly group class, like cycling or swimming, to increase your enjoyment and adherence to exercise.", "Weekly Goals": "Aim for 10,000 steps daily and include 3-4 exercise sessions per week."}, "Sleep & Well-being": {"Sleep Review": "An average of 6 hours of sleep per night might not be sufficient for your age, impacting your recovery and overall health.", "Suggestions": "Aim for at least 7-8 hours of sleep. Consider establishing a bedtime routine, reducing screen time an hour before bed, and trying relaxation techniques to improve sleep quality."}, "Weekly Summary": {"Summary": "This week, focus on increasing your fruit intake and activity levels while ensuring you get adequate sleep. Small adjustments can lead to significant health improvements.", "Goals": "Increase fruit consumption to 800g/week, achieve 10,000 steps daily, and get at least 7 hours of sleep consistently."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height": 178, "Weight": 85}, "Physical Activity": {"Weekly Steps": 7000, "Zone Minutes per week": 120, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 200, "Meat": 300, "Fruits": 300, "Vegetables": 400, "Grains": 500, "Nuts": 100, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The user’s diet is relatively balanced, with a good intake of vegetables and legumes. However, fruit and nut consumption is below optimal levels.", "Areas of Improvements": "Fruits should be increased to around 800g/week and nuts to around 150g/week for better nutrient intake.", "Suggested Meals": ["Breakfast: Greek yogurt with honey, berries, and a sprinkle of almonds.", "Lunch: Quinoa salad with chickpeas, spinach, cherry tomatoes, and a handful of walnuts.", "Dinner: Grilled salmon with a side of roasted vegetables and a mixed fruit salad for dessert."]}, "Physical Activity": {"Activity Assessment": "The user is moderately active with 7000 weekly steps and 120 zone minutes, which is good but could be improved.", "Zone Minutes and Intensity Feedback": "To enhance fitness, aim for at least 150 zone minutes per week through varied intensity workouts.", "Strength/Cardio Tips": "Incorporate two strength training sessions weekly to build muscle mass alongside cardio activities such as brisk walking or cycling.", "Weekly Goals": "Aim to increase weekly steps to at least 10,000 and add one additional exercise session."}, "Sleep & Well-being": {"Sleep Review": "The average sleep duration of 6 hours is lower than the recommended 7-9 hours for optimal health and recovery.", "Suggestions": "Establish a consistent sleep schedule, create a bedtime routine, and limit screen time before sleep to improve sleep quality."}, "Weekly Summary": {"Summary": "Current diet is solid, but fruit and nut intake needs improvement. Physical activity is adequate, but a slight increase is recommended to boost overall health.", "Goals": "Increase fruit and nut consumption, enhance physical activity levels, and improve sleep duration."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height": 178, "Weight": 85}, "Physical Activity": {"Weekly Steps": 6500, "Zone Minutes per week": 90, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 300, "Meat": 500, "Fruits": 200, "Vegetables": 400, "Grains": 1500, "Nuts": 50, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The user follows a Mediterranean diet, but the intake of fruits and nuts is relatively low compared to recommended levels.", "Areas of Improvements": "Fruits are consumed at 200g/week; it's advisable to increase this to around 800g/week. Nuts are also low at 50g/week, ideally aiming for 140g/week.", "Suggested Meals": ["Breakfast: Greek yogurt topped with mixed berries and a sprinkle of walnuts.", "Lunch: Quinoa salad with chickpeas, diced cucumber, cherry tomatoes, and a handful of almonds.", "Dinner: Grilled salmon with a side of steamed broccoli and a mixed greens salad drizzled with olive oil and lemon.", "Snacks: Hummus with carrot sticks and a handful of mixed nuts."]}, "Physical Activity": {"Activity Assessment": "The user has a moderate level of physical activity but could benefit from increasing exercise frequency and intensity.", "Zone Minutes and Intensity Feedback": "90 zone minutes per week indicate low-intensity activity; aiming for 150-300 minutes a week of moderate activity could improve cardiovascular health.", "Strength/Cardio Tips": "Incorporate strength training at least once a week to complement cardio and improve muscle mass and metabolism.", "Weekly Goals": "Increase weekly steps to a minimum of 8,000 and include at least three exercise sessions focusing on strength or higher intensity cardio."}, "Sleep & Well-being": {"Sleep Review": "An average of 6 hours of sleep per night is below the recommended level, which may affect overall health and recovery.", "Suggestions": "Aim for 7-9 hours of sleep per night. Implement a relaxing evening routine, limit screen time before bed, and consider mindfulness or light stretching to promote better sleep."}, "Weekly Summary": {"Summary": "The user is generally aligned with a Mediterranean diet but needs to enhance fruit and nut intake while improving physical activity levels and sleep duration.", "Goals": "1. Increase fruit intake to 800g/week. 2. Increase nut consumption to approximately 140g/week. 3. Boost weekly step count and exercise frequency."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height": 178, "Weight": 85}, "Physical Activity": {"Weekly Steps": 5500, "Zone Minutes": 90, "Exercise Sessions": 2}, "Sleep": {"Average Hours per Night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 300, "Meat": 400, "Fruits": 200, "Vegetables": 400, "Grains": 600, "Nuts": 100, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The current diet is somewhat balanced, but fruit intake is significantly low which can affect vitamin and mineral intake.", "Areas of Improvements": "Fruit consumption should be increased from 200g/week to around 800g/week to enhance overall nutrient intake.", "Suggested Meals": ["Breakfast: Greek yogurt with mixed berries and a sprinkle of nuts.", "Lunch: Quinoa salad with chickpeas, cherry tomatoes, cucumber, and feta cheese.", "Dinner: Grilled chicken with roasted vegetables and a side of a green salad.", "Snacks: Fresh fruit such as apples or oranges throughout the day."]}, "Physical Activity": {"Activity Assessment": "Current activity levels are moderate, but weekly steps can be improved to enhance cardiovascular health.", "Zone Minutes and Intensity Feedback": "90 zone minutes indicate some cardiovascular activity, but aim for more high-intensity workouts to reduce health risks.", "Strength/Cardio Tips": "Incorporate at least one more exercise session per week focusing on both strength training and high-intensity cardio.", "Weekly Goals": "Increase weekly steps to at least 8,000 and exercise sessions to 3 to improve overall fitness."}, "Sleep & Well-being": {"Sleep Review": "Average sleep duration of 6 hours is below the recommended 7-9 hours for adults, potentially impacting overall health and recovery.", "Suggestions": "Aim to establish a consistent bedtime routine, limit screen time before bed, and create a restful sleep environment."}, "Weekly Summary": {"Summary": "Overall, the user maintains a reasonably balanced diet with a focus on the Mediterranean diet, yet there are clear areas for improvement in fruit intake and physical fitness.", "Goals": "Increase fruit consumption, enhance physical activity levels, and improve sleep quality."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 32, "Height (cm)": 180, "Weight (kg)": 85}, "Physical Activity": {"Weekly Steps": 6000, "Zone Minutes per week": 90, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy (liters)": 1.5, "Legumes (grams)": 300, "Meat (grams)": 500, "Fruits (grams)": 400, "Vegetables (grams)": 600, "Grains (grams)": 800, "Nuts (grams)": 100, "Water (liters)": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The user's diet is reasonably aligned with Mediterranean principles but lacks sufficient fruits and nuts intake.", "Areas of Improvements": "Currently, the user consumes only 400g of fruits weekly, which should be increased to about 800g. Nuts intake is also low at 100g; aim for at least 200g.", "Suggested Meals": ["Breakfast: Greek yogurt topped with mixed berries and a sprinkle of almonds.", "Lunch: Quinoa salad with cherry tomatoes, cucumbers, feta cheese, and a handful of walnuts.", "Dinner: Grilled salmon with a side of steamed broccoli and a sweet potato.", "Snacks: Hummus with carrot and cucumber sticks, mixed nuts."]}, "Physical Activity": {"Activity Assessment": "User engages in moderate physical activity with 6000 steps and 90 zone minutes, which is below the recommended levels for cardiovascular fitness.", "Zone Minutes and Intensity Feedback": "Consider increasing zone minutes to at least 150 per week for better cardiovascular health.", "Strength/Cardio Tips": "Incorporate at least one more structured exercise session each week, focusing on both cardio and strength training.", "Weekly Goals": "Aim for a total of 10,000 steps daily and increase exercise sessions to at least 3 per week."}, "Sleep & Well-being": {"Sleep Review": "User averages 6.5 hours of sleep per night, which is below the recommended 7-9 hours for optimal health.", "Suggestions": "Establish a relaxing bedtime routine, limit screen time before bed, and aim for a consistent bedtime to improve sleep duration."}, "Weekly Summary": {"Summary": "The user's diet is largely Mediterranean, but improvements in fruit and nut intake are needed. Exercise levels are moderate but could benefit from more frequency and intensity.", "Goals": "Increase fruit and nut consumption, boost weekly physical activity to 3 sessions, and strive for better sleep quality."}}},
{"input": {"demographics": {"gender": "Male", "age": 35, "height_cm": 178, "weight_kg": 85}, "physical_activity": {"weekly_steps": 8000, "zone_minutes_per_week": 100, "exercise_sessions_per_week": 2}, "sleep": {"average_hours_per_night": 6.5}, "dietary_preference": {"declared_diet": "Mediterranean", "allergies": "no allergies"}, "weekly_food_consumption": {"dairy_liters": 1, "legumes_grams": 200, "meat_grams": 500, "fruits_grams": 400, "vegetables_grams": 600, "grains_grams": 800, "nuts_grams": 100, "water_liters": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "Your diet follows Mediterranean principles but is high in meat consumption while fruits and nuts are somewhat low.", "Areas of Improvements": "Increase your fruit intake to around 800g/week and nuts to 300g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with honey and mixed berries.", "Lunch: Quinoa salad with chickpeas, cherry tomatoes, cucumbers, and a sprinkle of nuts.", "Dinner: Grilled chicken with a side of stir-fried vegetables and a handful of almonds."]}, "Physical Activity": {"Activity Assessment": "You maintain a moderate level of physical activity but could benefit from more consistent exercise.", "Zone Minutes and Intensity Feedback": "Aim to increase your zone minutes to at least 150 per week by incorporating more vigorous activities.", "Strength/Cardio Tips": "Consider adding a third exercise session focused on strength training, such as bodyweight exercises or resistance training.", "Weekly Goals": "Aim for at least 10,000 steps per day and gradually increase your exercise sessions to 3 per week."}, "Sleep & Well-being": {"Sleep Review": "Your average sleep duration is slightly below the recommended amount for optimal health.", "Suggestions": "Try to aim for 7-8 hours of sleep per night, and consider establishing a relaxing bedtime routine to improve sleep quality."}, "Weekly Summary": {"Summary": "Overall, you have a solid foundation in your Mediterranean diet, yet there are areas for improvement in fruit and nut consumption. Increasing your physical activity to meet weekly goals and enhancing your sleep duration will benefit your overall well-being.", "Goals": "Focus on increasing fruit and nut intake, reach 10,000 steps daily, integrate more exercise sessions, and aim for better sleep habits."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height": 180, "Weight": 85}, "Physical Activity": {"Weekly Steps": 8000, "Zone Minutes": 90, "Exercise Sessions": 2}, "Sleep": {"Average Hours": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "No allergies"}, "Weekly Food Consumption": {"Dairy": 1, "Legumes": 300, "Meat": 700, "Fruits": 300, "Vegetables": 400, "Grains": 600, "Nuts": 100, "Water": 1.5}}, "output": {"Food Recommendation": {"Overall Assessment": "The user follows a Mediterranean diet, which is beneficial for heart health. However, the intake of fruits and nuts is lower than recommended.", "Areas of Improvements": "The user currently consumes only 300g of fruits per week; aim to increase intake to around 800g per week. Nuts are also consumed at a low level, suggesting an increase to at least 200g per week.", "Suggested Meals": ["Breakfast: Greek yogurt with mixed berries and a sprinkle of chopped nuts.", "Lunch: Quinoa salad with chickpeas, mixed greens, cherry tomatoes, cucumber, and olive oil dressing.", "Dinner: Grilled salmon with a side of steamed broccoli and sweet potato, topped with walnuts."]}, "Physical Activity": {"Activity Assessment": "The user is moderately active with an average of 8000 steps a week and 90 zone minutes.", "Zone Minutes and Intensity Feedback": "While the user engages in some physical activity, increasing intensity during workouts can enhance cardiovascular benefits.", "Strength/Cardio Tips": "Incorporate at least one additional strength training session per week to improve muscle mass and metabolism.", "Weekly Goals": "Aim for 10,000 steps daily and increase zone minutes to 150 per week."}, "Sleep & Well-being": {"Sleep Review": "The average sleep duration of 6.5 hours is below the recommended 7-9 hours for adults, which may impact overall well-being.", "Suggestions": "Establish a bedtime routine to enhance sleep quality, such as limiting screen time an hour before bed and creating a comfortable sleeping environment."}, "Weekly Summary": {"Summary": "The user is adhering to a healthy Mediterranean diet and maintaining reasonable physical activity levels but is falling short in fruit and nut consumption, along with sleep duration.", "Goals": "Increase fruit and nut intake this week, enhance physical activity intensity, and prioritize sleep to improve overall health."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 45, "Height": 175, "Weight": 85}, "Physical Activity": {"Weekly Steps": 5500, "Zone Minutes": 90, "Exercise Sessions": 2}, "Sleep": {"Average Hours": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1, "Legumes": 200, "Meat": 400, "Fruits": 300, "Vegetables": 500, "Grains": 1000, "Nuts": 100, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "Your diet adheres to the Mediterranean style, but you may benefit from more fruits and variety in legumes.", "Areas of Improvements": "Consider increasing your fruit intake from 300g/week to around 800g/week for better nutrition.", "Suggested Meals": ["Breakfast: Greek yogurt with honey and mixed berries.", "Lunch: Quinoa salad with chickpeas, cucumbers, tomatoes, and feta cheese.", "Dinner: Grilled salmon with a side of sautéed spinach and a mixed vegetable stir-fry."]}, "Physical Activity": {"Activity Assessment": "Your step count is below the recommended daily average, and your exercise sessions could be increased for better health.", "Zone Minutes and Intensity Feedback": "90 zone minutes indicate moderate activity; aim for at least 150 minutes per week.", "Strength/Cardio Tips": "Incorporate a mix of strength training and cardio, such as resistance exercises twice a week and brisk walking or cycling.", "Weekly Goals": "Aim to increase your steps to at least 8,000 daily and add one more exercise session per week."}, "Sleep & Well-being": {"Sleep Review": "Your average sleep duration is 6.5 hours, which is below the recommended duration for adults.", "Suggestions": "Aim for 7-9 hours of sleep per night. Consider establishing a relaxing bedtime routine and reducing screen time before sleep."}, "Weekly Summary": {"Summary": "You are following a Mediterranean diet with a good variety of foods, but there is a need for more fruits and physical activity.", "Goals": "Incorporate more fruits into your diet, increase your physical activity levels, and prioritize sleep to achieve better overall health."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 38, "Height": 180, "Weight": 85}, "Physical Activity": {"Weekly Steps": 5000, "Zone Minutes": 90, "Exercise Sessions": 2}, "Sleep": {"Average Hours": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 200, "Meat": 300, "Fruits": 400, "Vegetables": 600, "Grains": 800, "Nuts": 50, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "Your diet follows a Mediterranean pattern with a good intake of vegetables and grains but could use improvement in fruit and nut consumption.", "Areas of Improvements": "Increase fruit intake from 400g/week to around 800g/week and nuts from 50g/week to about 150g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with honey and mixed berries.", "Lunch: Quinoa salad with chickpeas, spinach, cherry tomatoes, and a handful of walnuts.", "Dinner: Grilled salmon with roasted vegetables and a side of mixed greens topped with sliced almonds.", "Snacks: A variety of fruits like apples, oranges, or bananas and a small handful of mixed nuts."]}, "Physical Activity": {"Activity Assessment": "Your current activity level is below the recommended amount for maintaining health and fitness.", "Zone Minutes and Intensity Feedback": "Consider increasing your zone minutes by engaging in moderate to vigorous activities like brisk walking or cycling for at least 150 minutes weekly.", "Strength/Cardio Tips": "Incorporate strength training exercises at least once a week to help build muscle mass.", "Weekly Goals": "Aim for at least 8,000 steps daily and increase your exercise sessions to 4 times a week."}, "Sleep & Well-being": {"Sleep Review": "You are currently averaging only 6 hours of sleep per night, which is below the recommended amount for adults. This could affect your overall health and recovery.", "Suggestions": "Try to aim for 7-8 hours of sleep each night. Establish a relaxing bedtime routine and avoid screens before bed to improve sleep quality."}, "Weekly Summary": {"Summary": "Overall, you have a good baseline diet and physical activity level, but there are several areas for improvement. Increasing your fruit and nut intake will enhance your diet quality, while boosting your physical activity and improving your sleep will help with overall well-being.", "Goals": "Increase fruit and nut consumption, meet step goals, enhance exercise frequency, and improve sleep duration."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 32, "Height": 178, "Weight": 82}, "Physical Activity": {"Weekly Steps": 5000, "Zone Minutes per week": 90, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 250, "Meat": 300, "Fruits": 400, "Vegetables": 600, "Grains": 700, "Nuts": 150, "Water": 1.8}}, "output": {"Food Recommendation": {"Overall Assessment": "Your diet reflects a reasonably good balance of food groups, but could use an increase in fruit intake.", "Areas of Improvements": "Currently consuming 400g of fruits per week, consider increasing it to around 800g/week for better nutrient intake.", "Suggested Meals": ["Breakfast: Greek yogurt with honey and a bowl of mixed berries.", "Lunch: Quinoa salad with chickpeas, diced tomatoes, cucumbers, and a variety of colorful bell peppers topped with olive oil.", "Dinner: Grilled chicken served with a side of sautéed spinach and roasted sweet potatoes.", "Snacks: Carrot sticks and hummus, or a fruit smoothie with banana and spinach."]}, "Physical Activity": {"Activity Assessment": "You are moderately active with a total of 5000 steps per week and two exercise sessions, which is good but can be improved.", "Zone Minutes and Intensity Feedback": "With 90 zone minutes per week, aim to increase this to 150-200 for better cardiovascular fitness.", "Strength/Cardio Tips": "Incorporate strength training exercises twice a week to complement your cardio sessions and improve muscle tone.", "Weekly Goals": "Increase your step count to at least 8000 steps and incorporate another exercise session to achieve a total of 3."}, "Sleep & Well-being": {"Sleep Review": "You are getting 6.5 hours of sleep, which is below the recommended 7-9 hours for optimal health.", "Suggestions": "Aim for a more consistent sleep schedule and create a relaxing bedtime routine to improve your quality of sleep."}, "Weekly Summary": {"Summary": "You maintain a balanced dietary approach with some room for improvement in fruit intake and physical activity levels. Sleep patterns also require attention.", "Goals": "1. Increase fruit consumption to 800g/week. 2. Raise step count to 8000/week. 3. Aim for 7-8 hours of sleep per night."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 42, "Height": 178, "Weight": 85}, "Physical Activity": {"Weekly Steps": 6500, "Zone Minutes": 90, "Exercise Sessions": 1}, "Sleep": {"Average Hours": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "No allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 300, "Meat": 800, "Fruits": 300, "Vegetables": 400, "Grains": 1000, "Nuts": 150, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "Your diet generally aligns with the Mediterranean style, but there's a moderate intake of fruits and vegetables, which could be improved.", "Areas of Improvements": "Increase your fruit intake from 300g/week to around 800g/week and vegetables from 400g/week to approximately 1000g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with mixed berries and a sprinkle of nuts.", "Lunch: Quinoa salad with chickpeas, mixed vegetables, and olive oil.", "Dinner: Grilled salmon with a side of roasted vegetables and brown rice.", "Snacks: Fresh fruit like apples or oranges, and carrot sticks with hummus."]}, "Physical Activity": {"Activity Assessment": "Your weekly steps and exercise sessions are below recommended levels for optimal health. Incorporating more physical activity can greatly benefit your well-being.", "Zone Minutes and Intensity Feedback": "Currently at 90 zone minutes per week, consider aiming for at least 150-200 minutes of moderate-intensity activity.", "Strength/Cardio Tips": "Incorporate strength training twice a week and try to increase your walking to at least 10,000 steps daily. Adding brisk walks or cycling can help.", "Weekly Goals": "Aim for at least 3 exercise sessions and 120 minutes of zone minutes this week."}, "Sleep & Well-being": {"Sleep Review": "At 6.5 hours per night, your sleep is slightly below the recommended duration for adults. Quality sleep is crucial for recovery and overall health.", "Suggestions": "Establish a bedtime routine to improve sleep quality, limit screen time before bed, and consider calming activities like reading or meditation."}, "Weekly Summary": {"Summary": "To enhance your health, focus on increasing your fruit and vegetable intake, improving physical activity levels, and enhancing sleep quality.", "Goals": "Increase fruit and vegetable servings, engage in regular exercise sessions, and aim for at least 7 hours of sleep each night."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 45, "Height": 178, "Weight": 90}, "Physical Activity": {"Weekly Steps": 6500, "Zone Minutes per week": 80, "Exercise Sessions per week": 1}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "No allergies"}, "Weekly Food Consumption": {"Dairy": 1, "Legumes": 300, "Meat": 600, "Fruits": 200, "Vegetables": 400, "Grains": 900, "Nuts": 100, "Water": 1.5}}, "output": {"Food Recommendation": {"Overall Assessment": "The user's diet includes a decent amount of legumes but lacks sufficient fruits and vegetables.", "Areas of Improvements": "Increase fruit intake from 200g to around 800g per week and vegetables from 400g to around 750g per week.", "Suggested Meals": ["Breakfast: Greek yogurt with honey and mixed berries.", "Lunch: Quinoa salad with chickpeas, spinach, cherry tomatoes, and a light vinaigrette.", "Dinner: Grilled salmon with steamed broccoli and a side of sweet potatoes.", "Snacks: A handful of nuts and an apple."]}, "Physical Activity": {"Activity Assessment": "Physical activity is low with only 80 zone minutes and 1 exercise session per week.", "Zone Minutes and Intensity Feedback": "Target at least 150 zone minutes per week to improve cardiovascular health.", "Strength/Cardio Tips": "Incorporate at least 2 additional exercise sessions per week focusing on both strength training and cardio.", "Weekly Goals": "Aim to increase weekly steps to 10,000 and include at least 150 minutes of moderate to vigorous activity."}, "Sleep & Well-being": {"Sleep Review": "Currently averaging only 6 hours of sleep, which is below the recommended amount for adults.", "Suggestions": "Establish a regular sleep schedule, reduce screen time before bed, and create a relaxing bedtime routine."}, "Weekly Summary": {"Summary": "The user maintains a Mediterranean diet but could improve intake of fruits and vegetables. Physical activity levels are low and sleep duration needs attention.", "Goals": "Increase fruit and vegetable consumption, enhance physical activity levels to at least 150 minutes, and improve sleep quality."}}},
{"input": {"Gender": "Male", "Age": 45, "Height": 178, "Weight": 85, "Physical Activity": {"Weekly Steps": 8500, "Zone Minutes per week": 90, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 200, "Meat": 400, "Fruits": 300, "Vegetables": 500, "Grains": 600, "Nuts": 100, "Water": 2.5}}, "output": {"Food Recommendation": {"Overall Assessment": "The user's diet exhibits a strong emphasis on vegetables and grains, typical of the Mediterranean diet, but the consumption of fruits and nuts is relatively low.", "Areas of Improvements": "Fruits should be increased to around 800g/week, and nuts should be increased to at least 200g/week to enhance nutrient diversity and improve heart health.", "Suggested Meals": ["Breakfast: Greek yogurt with honey and mixed berries topped with chopped almonds.", "Lunch: Quinoa salad with chickpeas, diced cucumber, bell peppers, and a drizzle of olive oil.", "Dinner: Grilled salmon with a side of sautéed spinach and roasted sweet potatoes, finished with a handful of walnuts.", "Snacks: A small handful of mixed nuts or an apple with almond butter."]}, "Physical Activity": {"Activity Assessment": "The user is moderately active with a decent number of steps and weekly exercise sessions, but there is room for improvement.", "Zone Minutes and Intensity Feedback": "With 90 zone minutes, the user is hitting a good level of moderate-intensity activity, but can benefit from increasing to 150 minutes weekly for additional health benefits.", "Strength/Cardio Tips": "Incorporate at least one more exercise session focused on strength training to improve muscle mass and metabolic health.", "Weekly Goals": "Aim for 10,000 steps per day and increase exercise sessions to 3-4 times per week, mixing cardio with strength training."}, "Sleep & Well-being": {"Sleep Review": "At an average of 6 hours per night, the user may not be getting sufficient rest, potentially impacting overall energy levels and health.", "Suggestions": "Aim for 7-9 hours of sleep per night. Create a relaxing bedtime routine and keep a consistent sleep schedule."}, "Weekly Summary": {"Summary": "The user maintains a generally healthy diet but should focus on increasing intake of fruits and nuts while enhancing physical activity and sleep quality.", "Goals": "Increase fruit intake to 800g/week, nuts to 200g/week, exercise sessions to 4 times per week, and sleep duration to at least 7 hours per night."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 35, "Height": 180, "Weight": 85}, "Physical Activity": {"Weekly Steps": 8000, "Zone Minutes per week": 120, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1, "Legumes": 500, "Meat": 300, "Fruits": 200, "Vegetables": 400, "Grains": 700, "Nuts": 100, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The current diet is relatively healthy, aligning with Mediterranean principles. However, fruit intake is lower than recommended.", "Areas of Improvements": "Fruits should be increased from 200g/week to around 800g/week for better nutrient diversity.", "Suggested Meals": ["Breakfast: Greek yogurt with honey and mixed berries.", "Lunch: Quinoa salad with chickpeas, diced cucumber, and cherry tomatoes.", "Snack: A handful of nuts and a banana.", "Dinner: Grilled chicken with a side of roasted vegetables and a mixed green salad."]}, "Physical Activity": {"Activity Assessment": "Moderate physical activity with a good balance of weekly steps and zone minutes.", "Zone Minutes and Intensity Feedback": "Aim for at least 150 minutes of moderate-intensity exercise weekly to improve cardiovascular health.", "Strength/Cardio Tips": "Incorporate strength training twice a week to enhance muscle mass alongside your cardio sessions.", "Weekly Goals": "Increase exercise sessions to 3 or 4 and target 10,000 steps daily for better fitness."}, "Sleep & Well-being": {"Sleep Review": "Getting 6 hours of sleep is below the recommended 7-9 hours for optimal health.", "Suggestions": "Aim to establish a bedtime routine and reduce screen time before sleep to improve sleep quality."}, "Weekly Summary": {"Summary": "Focus on increasing fruit consumption and adjusting sleep habits while maintaining current exercise levels.", "Goals": "Increase fruit intake, reach 10,000 daily steps, and aim for 7-8 hours of quality sleep."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 35, "Height": 178, "Weight": 85}, "Physical Activity": {"WeeklySteps": 7500, "ZoneMinutes": 100, "ExerciseSessions": 2}, "Sleep": {"AverageHours": 6}, "DietaryPreference": {"DeclaredDiet": "Mediterranean", "Allergies": "no allergies"}, "WeeklyFoodConsumption": {"Dairy": 1, "Legumes": 300, "Meat": 500, "Fruits": 400, "Vegetables": 600, "Grains": 1500, "Nuts": 150, "Water": 2}}, "output": {"Food Recommendation": {"OverallAssessment": "Your diet shows a good variety of food groups typical of a Mediterranean style, but your fruit intake is below the recommended levels.", "AreasOfImprovements": "Increase fruit consumption from 400g/week to around 800g/week.", "SuggestedMeals": ["Breakfast: Greek yogurt with honey and mixed berries.", "Lunch: Quinoa salad with chickpeas, cherry tomatoes, cucumber, and a lemon-olive oil dressing.", "Snack: A handful of almonds and an apple.", "Dinner: Grilled salmon with a side of roasted vegetables and a mixed green salad."]}, "Physical Activity": {"ActivityAssessment": "Your weekly step count and exercise sessions are moderate but can be improved for overall health.", "ZoneMinutesAndIntensityFeedback": "Aim to increase zone minutes to at least 150 to improve cardiovascular fitness.", "Strength/CardioTips": "Incorporate strength training at least once a week and consider increasing walking or cycling frequency.", "WeeklyGoals": "Aim for 10,000 steps daily and add one more exercise session this week."}, "Sleep & Well-being": {"SleepReview": "Your average sleep is below the recommended 7-9 hours for adults, which may impact your recovery and overall health.", "Suggestions": "Try to establish a consistent sleep schedule, limit screen time before bed, and create a relaxing bedtime routine."}, "WeeklySummary": {"Summary": "Your current diet and activity levels are decent, but there are clear areas for improvement, especially in fruit intake and sleep duration.", "Goals": "Increase fruit consumption, add more exercise sessions, and aim for better sleep hygiene."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 39, "Height": 178, "Weight": 85}, "Physical Activity": {"Weekly Steps": 7500, "Zone Minutes per week": 120, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1, "Legumes": 500, "Meat": 400, "Fruits": 200, "Vegetables": 400, "Grains": 600, "Nuts": 100, "Water": 1.5}}, "output": {"Food Recommendation": {"Overall Assessment": "The user follows a Mediterranean diet, which emphasizes healthy fats and whole foods, but is currently low in fruit intake.", "Areas of Improvements": "Fruits are consumed at only 200g/week; aim to increase to around 800g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with honey and mixed berries.", "Lunch: Quinoa salad with chickpeas, diced cucumber, tomatoes, and a lemon-olive oil dressing.", "Dinner: Grilled chicken with a side of roasted vegetables and a fresh fruit salad."]}, "Physical Activity": {"Activity Assessment": "Moderate activity levels with a good number of zone minutes, but could benefit from more exercise sessions.", "Zone Minutes and Intensity Feedback": "120 zone minutes indicate a decent intensity; consider adding more varied activities to enhance fitness.", "Strength/Cardio Tips": "Incorporate at least one additional exercise session weekly, focusing on a mix of strength training and cardio.", "Weekly Goals": "Increase to at least 3 exercise sessions per week."}, "Sleep & Well-being": {"Sleep Review": "With an average of 6.5 hours of sleep, the user might not be getting enough restorative sleep.", "Suggestions": "Aim to increase sleep duration to at least 7-8 hours by establishing a consistent bedtime routine."}, "Weekly Summary": {"Summary": "The user adheres to a Mediterranean diet but needs to increase fruit consumption and exercise frequency to improve overall health.", "Goals": "Increase fruit intake and exercise sessions to meet recommended guidelines."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height": 178, "Weight": 85}, "Physical Activity": {"Weekly Steps": 7500, "Zone Minutes per week": 120, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 300, "Meat": 600, "Fruits": 250, "Vegetables": 400, "Grains": 500, "Nuts": 150, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The user follows a Mediterranean diet, which is rich in healthy fats, fruits, vegetables, and whole grains. However, their fruit intake is relatively low.", "Areas of Improvements": "Fruits are currently consumed at 250g/week; it is recommended to increase this to around 800g/week for better nutrient balance.", "Suggested Meals": ["Breakfast: Greek yogurt topped with mixed berries and a drizzle of honey.", "Lunch: Quinoa salad with cherry tomatoes, cucumbers, feta cheese, and olives.", "Dinner: Grilled chicken with roasted vegetables and a side of fruit salad."]}, "Physical Activity": {"Activity Assessment": "The user's physical activity level is moderate, with 7500 steps and 120 zone minutes indicating a decent level of cardiovascular activity.", "Zone Minutes and Intensity Feedback": "120 zone minutes are within the recommended range for maintaining cardiovascular health, but increasing intensity or duration could yield greater benefits.", "Strength/Cardio Tips": "Incorporate 1 additional exercise session focused on strength training per week, alongside existing cardio activities.", "Weekly Goals": "Aim to increase weekly steps to at least 10,000 and add one more exercise session, focusing on strength."}, "Sleep & Well-being": {"Sleep Review": "Sleeping an average of 6 hours per night may not be sufficient for optimal health, leading to potential fatigue and lower physical performance.", "Suggestions": "Aim for at least 7-8 hours of sleep per night. Establish a calming bedtime routine and avoid screens at least an hour before bed."}, "Weekly Summary": {"Summary": "With a strong Mediterranean diet base, the user has good intake of healthy fats and vegetables but needs to boost fruit consumption and increase overall physical activity.", "Goals": "Increase fruit intake to 800g/week, add another exercise session, aim for 10,000 steps, and improve sleep duration to at least 7 hours."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 35, "Height": 180, "Weight": 78}, "Physical Activity": {"Weekly Steps": 6500, "Zone Minutes": 120, "Exercise Sessions": 2}, "Sleep": {"Average Hours": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 2, "Legumes": 200, "Meat": 400, "Fruits": 300, "Vegetables": 600, "Grains": 1000, "Nuts": 50, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The user's diet reflects a Mediterranean style with adequate vegetable intake but could benefit from an increase in fruits and nuts.", "Areas of Improvements": "Fruits consumption is currently at 300g/week; increasing it to around 800g/week would enhance nutrient variety. Nuts are also low at 50g/week, consider increasing to 100g/week for healthy fats.", "Suggested Meals": ["Breakfast: Greek yogurt with honey and mixed berries.", "Lunch: Quinoa salad with chickpeas, cherry tomatoes, cucumber, and a sprinkle of nuts.", "Dinner: Grilled salmon with a side of roasted vegetables and a mixed greens salad with olive oil dressing."]}, "Physical Activity": {"Activity Assessment": "The user maintains a reasonable level of physical activity but could improve on consistency and intensity.", "Zone Minutes and Intensity Feedback": "120 zone minutes per week is a good start; consider increasing to 150-180 for enhanced cardiovascular benefits.", "Strength/Cardio Tips": "Incorporate at least one strength training session weekly to build muscle. Try to include more brisk walking or jogging to raise heart rate during activities.", "Weekly Goals": "Aim for at least 10,000 steps daily and increase exercise sessions to 3 each week, incorporating both cardio and strength training."}, "Sleep & Well-being": {"Sleep Review": "The average sleep duration of 6.5 hours is below the recommended 7-9 hours for optimal health.", "Suggestions": "Establish a consistent bedtime routine, limit screen time before bed, and consider relaxation techniques such as meditation or deep breathing to improve sleep quality."}, "Weekly Summary": {"Summary": "The user displays a generally healthy Mediterranean diet but needs improvements in fruit and nut intake. Physical activity levels are decent, though more consistency is encouraged.", "Goals": ["Increase fruit intake to around 800g/week.", "Increase nut intake to 100g/week.", "Achieve 10,000 steps daily and increase exercise sessions to 3 per week."]}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height": 180, "Weight": 85}, "Physical Activity": {"Weekly Steps": 6000, "Zone Minutes per week": 120, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1, "Legumes": 500, "Meat": 300, "Fruits": 400, "Vegetables": 600, "Grains": 700, "Nuts": 100, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The user's diet is fairly balanced but lacks sufficient fruit and nut intake.", "Areas of Improvements": "Increase fruit intake from 400g/week to around 800g/week and nuts from 100g/week to at least 200g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with a mix of berries and a handful of nuts.", "Snack: Apple slices with almond butter.", "Lunch: Quinoa salad with chickpeas, cherry tomatoes, cucumber, and a handful of walnuts.", "Dinner: Grilled fish with a side of roasted vegetables and a mixed green salad topped with olive oil and lemon."]}, "Physical Activity": {"Activity Assessment": "User engages in moderate physical activity but may benefit from more frequent exercise sessions.", "Zone Minutes and Intensity Feedback": "Weekly zone minutes are good, but aim to increase intensity during workouts to improve cardiovascular fitness.", "Strength/Cardio Tips": "Incorporate strength training twice a week using body weight exercises or resistance bands to improve overall muscle strength.", "Weekly Goals": "Increase weekly steps to 8000 and add one more exercise session."}, "Sleep & Well-being": {"Sleep Review": "User averages 6 hours of sleep, which is below the recommended amount for adults.", "Suggestions": "Aim for 7-8 hours of sleep per night. Establish a calming bedtime routine and limit screen time before bed."}, "Weekly Summary": {"Summary": "User's diet is mostly healthy with some areas for improvement in fruit and nut consumption. Physical activity is moderate, but there is potential for growth.", "Goals": "Increase fruit and nut intake, add one more exercise session per week, and improve sleep duration to at least 7 hours."}}},
{"input": {"demographics": {"gender": "Male", "age": 37, "height_cm": 180, "weight_kg": 85}, "physical_activity": {"weekly_steps": 7500, "zone_minutes_per_week": 100, "exercise_sessions_per_week": 2}, "sleep": {"average_hours_per_night": 6.5}, "dietary_preference": {"declared_diet": "Mediterranean", "allergies": "no allergies"}, "weekly_food_consumption": {"dairy_liters": 2, "legumes_grams": 300, "meat_grams": 700, "fruits_grams": 400, "vegetables_grams": 600, "grains_grams": 800, "nuts_grams": 150, "water_liters": 2.5}}, "output": {"Food Recommendation": {"Overall Assessment": "The user's diet is fairly balanced, with a good intake of vegetables and grains, but fruit and nut consumption is on the lower side.", "Areas of Improvements": "Fruits consumption is low at 400g/week; aim to increase to around 800g/week. Nuts are also low at 150g/week; consider increasing to 250g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with mixed berries and a handful of walnuts.", "Lunch: Quinoa salad with chickpeas, diced bell peppers, and a drizzle of olive oil.", "Dinner: Grilled fish with a side of sautéed spinach and roasted sweet potatoes."]}, "Physical Activity": {"Activity Assessment": "The user is moderately active but could benefit from more consistent exercise sessions.", "Zone Minutes and Intensity Feedback": "100 zone minutes indicate some cardiovascular activity; aim to increase to 150 weekly to enhance fitness.", "Strength/Cardio Tips": "Incorporate strength training on non-cardio days to build muscle; consider activities like weight lifting or resistance training.", "Weekly Goals": "Aim for at least 3 exercise sessions/week, combining both cardio and strength training."}, "Sleep & Well-being": {"Sleep Review": "Average sleep duration is slightly below the recommended 7-9 hours for adults; quality and duration can impact overall health.", "Suggestions": "Establish a consistent sleep schedule, limit screen time before bed, and create a relaxing bedtime routine to enhance sleep quality."}, "Weekly Summary": {"Summary": "Overall, the user maintains a reasonably healthy lifestyle but has room for improvement in dietary variety and exercise frequency.", "Goals": "Increase fruit and nut intake, enhance exercise frequency to 3 sessions per week, and improve sleep quality."}}},
{"input": {"demographics": {"gender": "Male", "age": 32, "height_cm": 178, "weight_kg": 85}, "physical_activity": {"weekly_steps": 8000, "zone_minutes": 120, "exercise_sessions": 2}, "sleep": {"average_hours_per_night": 6}, "dietary_preference": {"declared_diet": "Mediterranean", "allergies": "no allergies"}, "weekly_food_consumption": {"dairy_liters": 1, "legumes_grams": 150, "meat_grams": 300, "fruits_grams": 400, "vegetables_grams": 200, "grains_grams": 600, "nuts_grams": 100, "water_liters": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "Your diet is moderately balanced with decent fruit and legume intake, but vegetables are on the lower end.", "Areas of Improvements": "Increase vegetable intake from 200g to around 800g per week to enhance nutrient diversity.", "Suggested Meals": ["Breakfast: Greek yogurt with chopped nuts and mixed berries.", "Lunch: Quinoa salad with mixed greens, chickpeas, and cherry tomatoes.", "Dinner: Grilled chicken with roasted vegetables and a side of hummus."]}, "Physical Activity": {"Activity Assessment": "You're engaging in a reasonable amount of physical activity with 8000 steps and 2 exercise sessions.", "Zone Minutes and Intensity Feedback": "120 zone minutes suggest a good mix of moderate to high-intensity workouts. Consider varying the intensity for better cardiovascular health.", "Strength/Cardio Tips": "Incorporate strength training at least once a week to balance your cardio efforts.", "Weekly Goals": "Aim to increase your weekly steps to 10,000 and include one additional exercise session."}, "Sleep & Well-being": {"Sleep Review": "Getting 6 hours of sleep is below the recommended guidelines for adults. This could impact overall health and recovery.", "Suggestions": "Aim for at least 7-8 hours of sleep per night. Consider establishing a relaxing bedtime routine and reducing screen time before bed."}, "Weekly Summary": {"Summary": "You have a good foundation with your Mediterranean diet, but improvements can be made by adding more vegetables and enhancing your sleep schedule.", "Goals": "1. Increase vegetable intake to 800g/week. 2. Add one more exercise session. 3. Improve sleep duration to 7 hours/night."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height": 180, "Weight": 85}, "Physical Activity": {"Weekly Steps": 7000, "Zone Minutes per week": 90, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 300, "Meat": 600, "Fruits": 200, "Vegetables": 400, "Grains": 700, "Nuts": 100, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The user's diet reflects some principles of the Mediterranean diet, but fruit intake is significantly below recommended levels.", "Areas of Improvements": "Fruits should be increased from 200g/week to around 800g/week to meet dietary recommendations.", "Suggested Meals": ["Breakfast: Greek yogurt with honey and mixed berries.", "Lunch: Quinoa salad with chickpeas, cherry tomatoes, cucumbers, and a drizzle of olive oil.", "Dinner: Grilled chicken with a side of steamed broccoli and sweet potatoes.", "Snacks: A handful of nuts and an apple."]}, "Physical Activity": {"Activity Assessment": "The user's physical activity is moderate, but additional exercise could enhance health benefits.", "Zone Minutes and Intensity Feedback": "Aim to increase zone minutes to 150 per week by incorporating more vigorous activities.", "Strength/Cardio Tips": "Include at least one more exercise session per week focused on strength training to build muscle.", "Weekly Goals": "Target 10,000 steps daily and ensure 4 exercise sessions per week."}, "Sleep & Well-being": {"Sleep Review": "The user averages 6 hours of sleep, which is below the recommended amount for optimal health.", "Suggestions": "Aim for at least 7-8 hours of sleep per night by establishing a bedtime routine and reducing screen time before bed."}, "Weekly Summary": {"Summary": "The user is making progress towards a Mediterranean diet but needs to increase fruit intake and exercise frequency.", "Goals": "Increase fruit consumption, enhance physical activity to 4 sessions weekly, and improve sleep quality to 7 hours nightly."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height (cm)": 178, "Weight (kg)": 85}, "Physical Activity": {"Weekly Steps": 8000, "Zone Minutes per week": 90, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy (liters)": 1.5, "Legumes (grams)": 300, "Meat (grams)": 500, "Fruits (grams)": 200, "Vegetables (grams)": 400, "Grains (grams)": 700, "Nuts (grams)": 100, "Water (liters)": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The user has a decent Mediterranean diet, but fruit intake is significantly low which is important for vitamins and fiber.", "Areas of Improvements": "Increase fruit consumption from 200g/week to around 800g/week. Also, consider increasing vegetable intake to diversify nutrients.", "Suggested Meals": ["Breakfast: Greek yogurt with mixed berries and a sprinkle of nuts.", "Lunch: Quinoa salad with chickpeas, cherry tomatoes, cucumbers, and avocado.", "Dinner: Grilled chicken with a side of roasted vegetables and a spinach salad.", "Snacks: Fresh fruit like apples or oranges, and a handful of almonds."]}, "Physical Activity": {"Activity Assessment": "The user walks a moderate amount but should aim for more consistent exercise to improve fitness and health.", "Zone Minutes and Intensity Feedback": "90 zone minutes indicate moderate activity; aim to increase this through more vigorous exercise sessions.", "Strength/Cardio Tips": "Incorporate a mix of strength training twice a week along with cardiovascular workouts to enhance fitness levels.", "Weekly Goals": "Aim to increase exercise sessions to 3-4 and hit closer to 150 zone minutes."}, "Sleep & Well-being": {"Sleep Review": "Getting an average of 6 hours per night may not be sufficient for optimal health; quality of sleep matters as well.", "Suggestions": "Establish a consistent bedtime routine, limit screen time before bed, and consider relaxation techniques such as meditation."}, "Weekly Summary": {"Summary": "To improve overall health, focus on increasing fruit and vegetable intake, boost physical activity levels, and improve sleep quality.", "Goals": "Increase fruit consumption, exercise more frequently, and enhance sleep duration to 7-8 hours per night."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height": 180, "Weight": 85}, "Physical Activity": {"Weekly Steps": 8000, "Zone Minutes per week": 120, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1, "Legumes": 200, "Meat": 500, "Fruits": 300, "Vegetables": 400, "Grains": 600, "Nuts": 100, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The diet shows a good balance of macronutrients with a significant intake of vegetables and grains, but fruit consumption is lower than optimal.", "Areas of Improvements": "Fruits are currently consumed at 300g/week; aiming for around 800g/week would enhance vitamin and fiber intake.", "Suggested Meals": ["Breakfast: Greek yogurt with honey and mixed berries.", "Lunch: Quinoa salad with chickpeas, cherry tomatoes, cucumber, and a lemon-olive oil dressing.", "Dinner: Grilled chicken with roasted vegetables and a side of brown rice.", "Snacks: Apple slices with almond butter or a handful of walnuts."]}, "Physical Activity": {"Activity Assessment": "Regular activity levels are present, but there is room for more frequent and varied exercise.", "Zone Minutes and Intensity Feedback": "Achieving 120 minutes weekly is good; aim for higher intensity during some sessions to improve cardiovascular fitness.", "Strength/Cardio Tips": "Incorporate strength training exercises twice a week and try to include 30 minutes of moderate to vigorous cardio in your routine.", "Weekly Goals": "Increase exercise sessions to 3-4 times a week and integrate more varied activities such as cycling or swimming."}, "Sleep & Well-being": {"Sleep Review": "6.5 hours of sleep is below the recommended amount for adults, which might affect energy levels and recovery.", "Suggestions": "Aim for 7-8 hours of sleep by setting a consistent bedtime and limiting screen time before bed."}, "Weekly Summary": {"Summary": "Overall, the current diet is healthy with room for improvement in fruit intake. Physical activity is adequate but can be boosted with additional sessions.", "Goals": {"Diet Goals": "Increase fruit intake to 800g/week and maintain a well-balanced diet.", "Activity Goals": "Exercise 3-4 times per week, with a mix of cardio and strength training.", "Sleep Goals": "Aim for 7-8 hours of restorative sleep each night."}}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height (cm)": 180, "Weight (kg)": 85}, "Physical Activity": {"Weekly Steps": 5500, "Zone Minutes per week": 90, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy (liters)": 1.5, "Legumes (grams)": 200, "Meat (grams)": 500, "Fruits (grams)": 300, "Vegetables (grams)": 400, "Grains (grams)": 600, "Nuts (grams)": 100, "Water (liters)": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "Your diet shows a good balance of Mediterranean principles, but you're consuming a lower amount of fruits and legumes.", "Areas of Improvements": "Aim to increase your fruit intake from 300g/week to around 800g/week and legumes from 200g/week to 500g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with mixed berries and a tablespoon of honey.", "Lunch: Quinoa salad with chickpeas, cherry tomatoes, cucumber, and a drizzle of olive oil.", "Dinner: Grilled salmon with a side of lentils and steamed broccoli."]}, "Physical Activity": {"Activity Assessment": "You are moderately active with a decent amount of weekly steps and exercise sessions.", "Zone Minutes and Intensity Feedback": "Aim to increase your zone minutes to at least 150 per week for better cardiovascular health.", "Strength/Cardio Tips": "Incorporate two strength training sessions and one cardio-focused workout per week.", "Weekly Goals": "Increase weekly steps to at least 8000 and try to include a structured workout routine."}, "Sleep & Well-being": {"Sleep Review": "Your average sleep duration is below recommended levels for optimal health and recovery.", "Suggestions": "Aim for 7-8 hours of sleep per night. Create a consistent bedtime routine and limit screen time before sleep."}, "Weekly Summary": {"Summary": "You're doing well with a Mediterranean diet but have opportunities to improve your fruit and legume intake, physical activity, and sleep quality.", "Goals": "Increase fruit and legume consumption, improve physical activity levels, and aim for better sleep hygiene."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 45, "Height (cm)": 175, "Weight (kg)": 85}, "Physical Activity": {"Weekly Steps": 6000, "Zone Minutes per week": 90, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy (liters)": 1, "Legumes (grams)": 300, "Meat (grams)": 600, "Fruits (grams)": 200, "Vegetables (grams)": 400, "Grains (grams)": 700, "Nuts (grams)": 50, "Water (liters)": 1.5}}, "output": {"Food Recommendation": {"Overall Assessment": "Your diet reflects some Mediterranean principles, but your fruit consumption is relatively low, and you consume a fair amount of meat.", "Areas of Improvements": "Increase fruit intake from 200g/week to around 800g/week, and consider reducing meat consumption while adding more legumes and nuts.", "Suggested Meals": ["Breakfast: Greek yogurt topped with mixed berries and a sprinkle of nuts.", "Lunch: Chickpea salad with tomatoes, cucumber, and lemon dressing.", "Dinner: Grilled vegetable and quinoa bowl with a side of fruit for dessert."]}, "Physical Activity": {"Activity Assessment": "You have a moderate level of physical activity but could benefit from increased frequency and intensity.", "Zone Minutes and Intensity Feedback": "Aim to increase your zone minutes by engaging in at least 150 minutes of moderate-intensity activities per week.", "Strength/Cardio Tips": "Incorporate strength training twice a week and aim for at least 10,000 steps daily.", "Weekly Goals": "Set a goal to increase your total weekly exercise sessions to 3-4 and include a variety of activities such as walking, cycling, or swimming."}, "Sleep & Well-being": {"Sleep Review": "You're averaging 6 hours of sleep, which is below the recommended amount for your age.", "Suggestions": "Aim for 7-8 hours of sleep per night by establishing a calming bedtime routine and limiting screen time before bed."}, "Weekly Summary": {"Summary": "Focus on improving your fruit intake and physical activity levels while ensuring you get enough sleep for overall well-being.", "Goals": "Increase fruit intake, boost physical activity to 10,000 steps daily, and establish a consistent sleep schedule."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 40, "Height": 180, "Weight": 85}, "Physical Activity": {"Weekly Steps": 6500, "Zone Minutes per week": 100, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 300, "Meat": 400, "Fruits": 200, "Vegetables": 400, "Grains": 600, "Nuts": 100, "Water": 1.5}}, "output": {"Food Recommendation": {"Overall Assessment": "The user follows a Mediterranean diet, but the intake of fruits is significantly low, which may affect overall nutrient intake.", "Areas of Improvements": "Fruits should be increased from 200g/week to around 800g/week to enhance vitamins and fiber intake.", "Suggested Meals": "Add a fruit salad for breakfast, incorporate smoothies with berries as snacks, and add fruits like apples or bananas to lunch and dinner."}, "Physical Activity": {"Activity Assessment": "The user engages in a moderate amount of physical activity but could benefit from increasing both the number of weekly exercise sessions and overall intensity.", "Zone Minutes and Intensity Feedback": "While 100 zone minutes per week is reasonable, aiming for at least 150 minutes would be ideal for improved cardiovascular health.", "Strength/Cardio Tips": "Incorporate short, high-intensity interval training (HIIT) sessions twice per week to boost cardiovascular fitness and strength training.", "Weekly Goals": "Aim to increase weekly exercise sessions to 4 and increase total steps to at least 10,000 per day."}, "Sleep & Well-being": {"Sleep Review": "The user averages 6 hours of sleep per night, which is below the recommended amount for optimal health.", "Suggestions": "Establish a consistent sleep schedule, limit screen time before bed, and create a relaxing bedtime routine to improve the quality and duration of sleep."}, "Weekly Summary": {"Summary": "The user is largely adhering to a Mediterranean diet but has notable gaps in fruit consumption. Increased physical activity and better sleep hygiene are also recommended.", "Goals": "Increase fruit intake, improve exercise frequency, enhance sleep quality, and aim for 10,000 steps daily."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 42, "Height": 180, "Weight": 95}, "Physical Activity": {"Weekly Steps": 6000, "Zone Minutes per week": 110, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 2, "Legumes": 300, "Meat": 600, "Fruits": 200, "Vegetables": 500, "Grains": 700, "Nuts": 50, "Water": 1.5}}, "output": {"Food Recommendation": {"Overall Assessment": "The current diet features a decent intake of vegetables and legumes, but fruit consumption is low and meat intake is higher than recommended for a Mediterranean diet.", "Areas of Improvements": "Consider increasing fruit consumption from 200g to around 800g per week and reducing meat intake to align closer with Mediterranean dietary guidelines.", "Suggested Meals": ["Breakfast: Greek yogurt with mixed berries and a sprinkle of nuts.", "Lunch: Quinoa salad with chickpeas, cherry tomatoes, cucumbers, and a olive oil dressing.", "Dinner: Grilled fish served with a side of sautéed spinach and roasted sweet potatoes."]}, "Physical Activity": {"Activity Assessment": "The user is moderately active but could benefit from increased physical activity to improve overall fitness and health.", "Zone Minutes and Intensity Feedback": "Aiming for at least 150 minutes of moderate exercise per week would be beneficial. Current zone minutes of 110 suggest some room for improvement.", "Strength/Cardio Tips": "Incorporate two sessions of strength training a week, focusing on compound movements, alongside cardio sessions that could include brisk walking or cycling.", "Weekly Goals": "Aim to increase weekly steps to at least 10,000 and schedule one additional exercise session (strength or cardio)."}, "Sleep & Well-being": {"Sleep Review": "With an average of 6 hours of sleep per night, the user is not meeting the recommended 7-9 hours, which may affect overall health and recovery.", "Suggestions": "Establish a relaxing bedtime routine, reduce screen time before bed, and aim for consistent sleep and wake times to improve sleep quality."}, "Weekly Summary": {"Summary": "The user has a foundation of healthy eating and physical activity but should focus on increasing fruit intake, engaging in more physical activity, and improving sleep quality.", "Goals": "In the coming week, aim for 800g of fruits, adjust meat consumption, increase exercise sessions from 2 to 3, and work towards getting at least 7 hours of sleep each night."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 35, "Height": 180, "Weight": 85}, "Physical Activity": {"Weekly Steps": 9500, "Zone Minutes per week": 120, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 300, "Meat": 600, "Fruits": 500, "Vegetables": 400, "Grains": 800, "Nuts": 100, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The diet is fairly balanced, leaning towards Mediterranean principles. However, consumption of fruits and vegetables could be higher to enhance nutrient intake.", "Areas of Improvements": "Fruits could be increased to around 800g/week and vegetables to around 600g/week for better health benefits.", "Suggested Meals": ["Greek yogurt with mixed berries and a sprinkle of walnuts for breakfast.", "Chickpea salad with cucumbers, tomatoes, and a drizzle of olive oil for lunch.", "Stir-fried mixed vegetables with lentils and brown rice for dinner."]}, "Physical Activity": {"Activity Assessment": "Weekly physical activity is moderate but could be improved by incorporating more structured exercise sessions.", "Zone Minutes and Intensity Feedback": "120 zone minutes indicate a good level of cardiovascular activity. Aim to increase to 150 minutes for optimal heart health.", "Strength/Cardio Tips": "Include at least one session of strength training per week to build muscle mass and support metabolism.", "Weekly Goals": "Increase exercise sessions to 3 per week and aim for a total of 150 zone minutes."}, "Sleep & Well-being": {"Sleep Review": "Average sleep duration is below the recommended 7-9 hours; insufficient rest can impact overall health.", "Suggestions": "Consider establishing a consistent bedtime routine and limit screen time before bed to improve sleep quality."}, "Weekly Summary": {"Summary": "Overall, the user has a balanced Mediterranean diet with room for improvement in fruit and vegetable intake. Physical activity is moderate but could be enhanced, and sleep duration needs attention.", "Goals": "Increase fruit and vegetable consumption, add more exercise sessions, and improve sleep duration to 7-8 hours per night."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height": 182, "Weight": 81}, "Physical Activity": {"Weekly Steps": 6500, "Zone Minutes": 90, "Exercise Sessions": 2}, "Sleep": {"Average Hours": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 200, "Meat": 350, "Fruits": 400, "Vegetables": 300, "Grains": 600, "Nuts": 150, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The diet reflects a decent Mediterranean approach; however, fruits and vegetables could be increased for better nutrient diversity.", "Areas of Improvements": "Fruits should be increased to around 800g/week, and vegetables should be increased to at least 500g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with honey and a variety of fruits (berries and banana).", "Lunch: Quinoa salad with chickpeas, cucumbers, tomatoes, and a drizzle of olive oil.", "Dinner: Grilled fish served with a side of steamed broccoli and sweet potatoes."]}, "Physical Activity": {"Activity Assessment": "Activity levels are moderate, but more consistency in exercise sessions would be beneficial.", "Zone Minutes and Intensity Feedback": "90 Zone Minutes indicates low intensity; consider increasing intensity during workouts to improve cardiovascular health.", "Strength/Cardio Tips": "Incorporate at least one session of strength training per week and aim for 150 minutes of moderate aerobic activity.", "Weekly Goals": "Aim to increase weekly steps to at least 8000 and include 3 exercise sessions."}, "Sleep & Well-being": {"Sleep Review": "Average sleep duration is 6.5 hours, which is below the recommended 7-9 hours for adults.", "Suggestions": "Establish a calming bedtime routine, minimize screen time before bed, and consider a consistent sleep schedule."}, "Weekly Summary": {"Summary": "The user follows a Mediterranean diet but has room for improvement in fruit and vegetable intake. Physical activity is moderate but could benefit from adjustments in intensity and frequency.", "Goals": "Increase weekly fruit and vegetable intake, enhance physical activity routine to include more exercise sessions, and prioritize sleep hygiene for improved rest."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height": 178, "Weight": 85}, "Physical Activity": {"Weekly Steps": 8000, "Zone Minutes per week": 90, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 300, "Meat": 700, "Fruits": 400, "Vegetables": 600, "Grains": 400, "Nuts": 100, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The diet is relatively balanced but could benefit from an increase in fruits and nuts, while the meat intake is on the higher side for a Mediterranean diet.", "Areas of Improvements": "Currently consuming 400g/week of fruits, it is recommended to increase this to around 800g/week. Additionally, the nut intake should be increased to at least 200g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with mixed berries and a handful of walnuts.", "Lunch: Quinoa salad with chickpeas, diced tomatoes, cucumbers, and a drizzle of olive oil.", "Dinner: Grilled salmon with steamed broccoli and a side of mixed greens topped with sliced almonds."]}, "Physical Activity": {"Activity Assessment": "Weekly physical activity is moderate but could be increased to improve overall health and fitness.", "Zone Minutes and Intensity Feedback": "Currently at 90 zone minutes per week, consider increasing to at least 150 minutes to enhance cardiovascular health.", "Strength/Cardio Tips": "Incorporate strength training exercises twice a week and aim for at least 30 minutes of moderate-intensity aerobic exercise most days.", "Weekly Goals": "Aim for a total of 150 zone minutes and 3-4 exercise sessions this week."}, "Sleep & Well-being": {"Sleep Review": "With an average of 6 hours per night, sleep duration is below the recommended levels for adults, which can affect overall health.", "Suggestions": "Try to aim for at least 7-8 hours of sleep per night. Create a relaxing bedtime routine and limit screen time before bed."}, "Weekly Summary": {"Summary": "This week, focus on increasing fruit and nut intake, enhancing physical activity levels, and prioritizing better sleep habits.", "Goals": "Increase fruit and nut consumption, aim for 150 zone minutes of exercise, and improve sleep duration to 7-8 hours."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 32, "Height": 178, "Weight": 82}, "Physical Activity": {"Weekly Steps": 6500, "Zone Minutes per week": 110, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.2, "Legumes": 300, "Meat": 600, "Fruits": 400, "Vegetables": 500, "Grains": 1000, "Nuts": 150, "Water": 2.5}}, "output": {"Food Recommendation": {"Overall Assessment": "The user's diet reflects a reasonable balance, particularly with legumes, vegetables, and grains. However, fruit intake is below recommended levels.", "Areas of Improvements": "Fruits should be increased from 400g/week to around 800g/week to meet recommended intake.", "Suggested Meals": ["Breakfast: Greek yogurt with mixed berries and a sprinkle of nuts.", "Lunch: Quinoa salad with mixed vegetables and chickpeas, drizzled with olive oil.", "Dinner: Grilled salmon with a side of roasted sweet potatoes and a spinach salad."]}, "Physical Activity": {"Activity Assessment": "The user is moderately active but could benefit from increased exercise frequency.", "Zone Minutes and Intensity Feedback": "110 zone minutes indicate a fair amount of moderate to intense activity; aim to increase intensity or duration.", "Strength/Cardio Tips": "Incorporate strength training sessions twice a week and consider adding a weekly cardio workout.", "Weekly Goals": "Aim for 10,000 steps daily and 150 minutes of moderate activity per week."}, "Sleep & Well-being": {"Sleep Review": "The user is getting slightly less than the recommended amount of sleep, which may affect overall health and recovery.", "Suggestions": "Try to increase sleep duration to at least 7-8 hours. Establish a calming bedtime routine and limit screen time before bed."}, "Weekly Summary": {"Summary": "The user maintains a balanced diet with room for improvement in fruit intake. Physical activity levels are satisfactory but could be enhanced with more consistency in exercise.", "Goals": "Increase fruit consumption, achieve 10,000 steps daily, and improve sleep quality."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height": 178, "Weight": 85}, "Physical Activity": {"Weekly Steps": 5000, "Zone Minutes per week": 90, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 300, "Meat": 700, "Fruits": 350, "Vegetables": 400, "Grains": 600, "Nuts": 100, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "Your diet adheres to the Mediterranean principles but shows room for improvement in fruit and nut intake.", "Areas of Improvements": "Increase fruit consumption from 350g/week to around 800g/week. Aim to consume more nuts, increasing from 100g/week to about 200g/week.", "Suggested Meals": ["Breakfast: Greek yogurt topped with mixed berries and a sprinkle of walnuts.", "Lunch: Quinoa salad with chickpeas, cherry tomatoes, cucumber, and a handful of almonds.", "Dinner: Grilled chicken with a side of roasted vegetables and a mixed green salad drizzled with olive oil.", "Snacks: Fresh fruit like apples or oranges, and a handful of mixed nuts."]}, "Physical Activity": {"Activity Assessment": "You are moderately active, but increasing your weekly exercise sessions could further enhance your fitness.", "Zone Minutes and Intensity Feedback": "Your zone minutes indicate a moderate level of intensity; consider incorporating higher intensity intervals to improve cardiovascular health.", "Strength/Cardio Tips": "Include strength training exercises at least once a week and aim for more cardio sessions, such as brisk walking or cycling, to reach 150+ minutes per week.", "Weekly Goals": "Aim to increase your weekly steps to 8000 and exercise sessions to 3."}, "Sleep & Well-being": {"Sleep Review": "You are getting slightly less than the recommended amount of sleep. Prioritize getting at least 7-8 hours.", "Suggestions": "Establish a consistent sleep schedule, limit screen time before bed, and create a calming bedtime routine to improve sleep quality."}, "Weekly Summary": {"Summary": "You are following a primarily Mediterranean diet, which is beneficial for health. However, increasing your fruit and nut consumption can offer additional health benefits. Expanding your physical activity can also enhance your fitness levels.", "Goals": "Increase fruit and nut intake, enhance physical activity levels, and improve sleep quality."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 45, "Height": 178, "Weight": 85}, "Physical Activity": {"Weekly Steps": 6500, "Zone Minutes": 100, "Exercise Sessions": 2}, "Sleep": {"Average Hours": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "No allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 200, "Meat": 600, "Fruits": 300, "Vegetables": 400, "Grains": 700, "Nuts": 100, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The user has a reasonably balanced diet with a good variety of grains and vegetables. However, fruit intake is below ideal levels for a Mediterranean diet.", "Areas of Improvements": "Fruits should be increased from 300g/week to around 800g/week to better align with the Mediterranean dietary guidelines.", "Suggested Meals": ["Breakfast: Greek yogurt with honey and mixed berries.", "Lunch: Quinoa salad with chickpeas, cucumber, tomatoes, and a squeeze of lemon.", "Dinner: Grilled salmon with a side of steamed broccoli and brown rice."]}, "Physical Activity": {"Activity Assessment": "Activity levels are moderate but could be improved. The user is below the recommended 150 minutes of moderate exercise per week.", "Zone Minutes and Intensity Feedback": "100 zone minutes indicate light activity; consider increasing intensity during workouts for better cardiovascular benefits.", "Strength/Cardio Tips": "Incorporate strength training twice a week and aim for brisk walking or jogging to boost cardiovascular health.", "Weekly Goals": "Increase weekly steps to 10,000 and add one additional exercise session."}, "Sleep & Well-being": {"Sleep Review": "Average sleep duration is slightly below the recommended 7-9 hours for adults. This can affect overall well-being.", "Suggestions": "Establish a regular bedtime routine, limit screen time before bed, and avoid caffeine in the evening to improve sleep quality."}, "Weekly Summary": {"Summary": "The user has a good foundation in terms of diet with opportunities for improvement in fruit intake and activity levels.", "Goals": "Increase fruit intake to 800g/week, enhance activity to 10,000 steps, and aim for 3 exercise sessions per week."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height": 178, "Weight": 85}, "Physical Activity": {"Weekly Steps": 8000, "Zone Minutes per week": 120, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 300, "Meat": 500, "Fruits": 400, "Vegetables": 600, "Grains": 700, "Nuts": 150, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "Your diet shows a decent variety of food groups, particularly with legumes and vegetables. However, your fruit intake is lower than recommended for optimal health.", "Areas of Improvements": "Currently consuming 400g/week of fruits, aim to increase this to around 800g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with mixed berries and a sprinkle of nuts.", "Lunch: Quinoa salad with chickpeas, cherry tomatoes, cucumber, and a lemon vinaigrette.", "Dinner: Grilled chicken with a side of roasted vegetables and a fresh fruit salad."]}, "Physical Activity": {"Activity Assessment": "You are moderately active with an average of 8000 steps and 2 exercise sessions per week.", "Zone Minutes and Intensity Feedback": "Great job on achieving 120 zone minutes! Try to increase this to 150 for improved cardiovascular fitness.", "Strength/Cardio Tips": "Incorporate a mix of strength training and high-intensity intervals in your workouts to boost overall fitness and efficiency.", "Weekly Goals": "Aim for at least 10,000 steps daily and add one more exercise session per week."}, "Sleep & Well-being": {"Sleep Review": "You're getting an average of 6.5 hours of sleep which is slightly below the recommended amount for adults.", "Suggestions": "Try to establish a bedtime routine and limit screen time before bed to improve sleep quality. Aim for at least 7-8 hours of sleep."}, "Weekly Summary": {"Summary": "Overall, your diet is balanced but could benefit from increased fruit intake. Your activity levels are good, but you can push for more intensity and frequency in workouts.", "Goals": "Increase fruit consumption, aim for higher activity levels, and improve sleep duration."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height": 180, "Weight": 85}, "Physical Activity": {"Weekly Steps": 6500, "Zone Minutes per week": 90, "Exercise Sessions per week": 1}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 300, "Meat": 500, "Fruits": 400, "Vegetables": 600, "Grains": 1000, "Nuts": 150, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The diet shows a good variety of foods, but the intake of fruits and nuts is below recommended levels for a Mediterranean diet.", "Areas of Improvements": "Increase fruit consumption from 400g/week to around 800g/week and nuts from 150g/week to at least 300g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with mixed berries and a handful of almonds.", "Lunch: Quinoa salad with chickpeas, diced bell peppers, cherry tomatoes, and a drizzle of olive oil.", "Dinner: Grilled fish with roasted vegetables and a side of couscous mixed with walnuts.", "Snacks: Hummus with carrot sticks and a handful of walnuts."]}, "Physical Activity": {"Activity Assessment": "Physical activity levels are moderate but could be improved for better health outcomes.", "Zone Minutes and Intensity Feedback": "90 Zone Minutes indicate some engagement, but aiming for at least 150 per week would be beneficial.", "Strength/Cardio Tips": "Incorporate at least 2 strength training sessions per week along with moderate-intensity cardio.", "Weekly Goals": "Aim to increase total weekly steps to around 10,000 and add at least 2 more exercise sessions."}, "Sleep & Well-being": {"Sleep Review": "With an average of 6 hours of sleep, rest is below the recommended 7-9 hours, which may affect overall health.", "Suggestions": "Consider establishing a bedtime routine to promote better sleep quality, such as reducing screen time an hour before bed and creating a calming environment."}, "Weekly Summary": {"Summary": "The current diet is diverse but lacks adequate fruits and nuts, while physical activity is at a moderate level with room for improvement. Sleep duration is short, impacting overall well-being.", "Goals": "1. Increase fruit and nut intake. 2. Improve physical activity levels by adding more workouts and increasing step count. 3. Aim for at least 7 hours of quality sleep each night."}}},
{"input": {"demographics": {"gender": "Male", "age": 34, "height_cm": 180, "weight_kg": 85}, "physical_activity": {"weekly_steps": 7500, "zone_minutes": 90, "exercise_sessions": 2}, "sleep": {"average_hours_per_night": 6.5}, "dietary_preference": {"declared_diet": "Mediterranean", "allergies": "no allergies"}, "weekly_food_consumption": {"dairy_liters": 1.5, "legumes_grams": 300, "meat_grams": 500, "fruits_grams": 400, "vegetables_grams": 200, "grains_grams": 800, "nuts_grams": 100, "water_liters": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The user's diet is relatively balanced with a good intake of grains and legumes, but fruits and vegetables are below recommended levels.", "Areas of Improvements": "Increase vegetables to around 500g/week and fruits to around 800g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with honey and mixed berries.", "Lunch: Quinoa salad with chickpeas, diced cucumber, and cherry tomatoes.", "Dinner: Grilled chicken with a side of roasted vegetables and a small serving of brown rice."]}, "Physical Activity": {"Activity Assessment": "The user is moderately active but could benefit from increasing their exercise sessions.", "Zone Minutes and Intensity Feedback": "Currently achieving lower zone minutes, consider incorporating more high-intensity workouts.", "Strength/Cardio Tips": "Engage in strength training 1-2 times a week, focusing on major muscle groups. Incorporate at least 30 minutes of cardio on non-strength days.", "Weekly Goals": "Aim for 10,000 steps a day and increase exercise sessions to 3."}, "Sleep & Well-being": {"Sleep Review": "The average sleep duration is slightly below the recommended 7-9 hours. This can impact overall health and well-being.", "Suggestions": "Establish a calming pre-sleep routine, aim to go to bed and wake up at the same time each day, and limit screen time before bed."}, "Weekly Summary": {"Summary": "The user has a solid foundation in their diet and physical activity but needs to enhance fruit and vegetable intake and improve sleep duration.", "Goals": "Increase physical activity sessions, boost fruit and vegetable intake, and optimize sleep duration."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 35, "Height": 178, "Weight": 85}, "Physical Activity": {"Weekly Steps": 8500, "Zone Minutes": 120, "Exercise Sessions": 2}, "Sleep": {"Average Hours per Night": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "No allergies"}, "Weekly Food Consumption": {"Dairy": 2, "Legumes": 300, "Meat": 600, "Fruits": 250, "Vegetables": 400, "Grains": 700, "Nuts": 150, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "Your current diet is relatively well-rounded, but there is room for improvement in fruit and vegetable intake.", "Areas of Improvements": "Increase fruit consumption from 250g/week to around 800g/week and vegetable intake from 400g/week to around 1000g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with honey and mixed berries.", "Lunch: Quinoa salad with chickpeas, cucumbers, tomatoes, and a variety of colorful veggies.", "Dinner: Grilled salmon with steamed broccoli and sweet potatoes.", "Snacks: A handful of almonds and an apple."]}, "Physical Activity": {"Activity Assessment": "You are moderately active with a good number of zone minutes, but there is potential to engage in more exercise sessions.", "Zone Minutes and Intensity Feedback": "Expanding your exercise to 3-4 sessions a week could improve your fitness levels, aiming for at least 150 zone minutes.", "Strength/Cardio Tips": "Incorporate both strength training and cardio exercises; aim for at least one strength session to build muscle.", "Weekly Goals": "Increase weekly exercise sessions to 3 and aim for at least 150 total zone minutes."}, "Sleep & Well-being": {"Sleep Review": "Your average sleep duration is below the recommended 7-9 hours, which could impact your overall health and recovery.", "Suggestions": "Aim to increase sleep duration by establishing a bedtime routine, reducing screen time before bed, and creating a calming sleep environment."}, "Weekly Summary": {"Summary": "Focus on increasing fruit and vegetable intake while maintaining a balanced diet. Add more exercise sessions and prioritize sleep.", "Goals": {"Food": "Increase fruit and vegetable consumption.", "Physical Activity": "Exercise 3 times weekly.", "Sleep": "Aim for 7-8 hours of sleep nightly."}}}},
{"input": {"demographics": {"gender": "Male", "age": 34, "height_cm": 178, "weight_kg": 85}, "physical_activity": {"weekly_steps": 8000, "zone_minutes": 120, "exercise_sessions": 2}, "sleep": {"average_hours_per_night": 6}, "dietary_preference": {"declared_diet": "Mediterranean", "allergies": "no allergies"}, "weekly_food_consumption": {"dairy_liters": 1, "legumes_grams": 500, "meat_grams": 200, "fruits_grams": 300, "vegetables_grams": 400, "grains_grams": 1000, "nuts_grams": 100, "water_liters": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "Your diet follows Mediterranean principles, but fruit intake is below recommended levels. Increasing your fruit consumption can boost antioxidants and fiber.", "Areas of Improvements": "Aim to increase fruit consumption to around 800g/week, while maintaining your current intake of vegetables and legumes.", "Suggested Meals": ["Breakfast: Greek yogurt with a mix of berries and a handful of nuts.", "Lunch: Quinoa salad with chickpeas, chopped vegetables, and a vinaigrette.", "Dinner: Grilled chicken with a side of roasted sweet potatoes and steamed broccoli."]}, "Physical Activity": {"Activity Assessment": "You're moderately active with a good number of weekly steps and zone minutes. However, increasing exercise sessions could enhance your fitness level.", "Zone Minutes and Intensity Feedback": "120 zone minutes are adequate but consider incorporating more vigorous activities to improve cardiovascular health.", "Strength/Cardio Tips": "Include at least one strength training session per week to enhance muscle mass and metabolic rate.", "Weekly Goals": "Aim for 10,000 steps daily and add an additional exercise session to your week."}, "Sleep & Well-being": {"Sleep Review": "6 hours of sleep is slightly below the recommended amount for optimal health.", "Suggestions": "Consider establishing a consistent bedtime routine and reducing screen time before bed to improve sleep quality."}, "Weekly Summary": {"Summary": "Your Mediterranean diet is solid, but enhancing fruit intake and improving sleep duration would benefit overall health. Regular physical activity is good, but further engagement in exercise sessions will support fitness goals.", "Goals": "Increase fruit consumption to 800g/week, add an exercise session, and aim for 7-8 hours of sleep each night."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height": 178, "Weight": 85}, "Physical Activity": {"Weekly Steps": 7000, "Zone Minutes": 90, "Exercise Sessions": 2}, "Sleep": {"Average Hours": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "No allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 300, "Meat": 600, "Fruits": 300, "Vegetables": 400, "Grains": 700, "Nuts": 100, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The user's diet shows a decent variety, particularly in legumes and vegetables, but there is a lack of fruits and nuts.", "Areas of Improvements": "Increase fruit intake from 300g/week to around 800g/week and nuts from 100g/week to about 200g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with mixed berries and a handful of almonds.", "Lunch: Quinoa salad with cherry tomatoes, cucumbers, chickpeas, and a drizzle of olive oil.", "Dinner: Grilled chicken with roasted vegetables and a side of spinach salad with walnuts.", "Snacks: Sliced apples with peanut butter or a small serving of trail mix."]}, "Physical Activity": {"Activity Assessment": "The user is moderately active but could benefit from increasing their weekly exercise sessions.", "Zone Minutes and Intensity Feedback": "Currently at 90 minutes/week, aim to reach 150 minutes/week for better cardiovascular health.", "Strength/Cardio Tips": "Incorporate strength training twice a week and consider walking for 30 minutes daily to increase step count.", "Weekly Goals": "Aim for 12,000 steps per week and increase exercise sessions to at least 3."}, "Sleep & Well-being": {"Sleep Review": "At 6.5 hours per night, the user is slightly below the recommended amount for adults, which may affect overall well-being.", "Suggestions": "Aim for at least 7-8 hours of sleep. Establish a bedtime routine, avoid screens before bed, and create a relaxing sleep environment."}, "Weekly Summary": {"Summary": "This user is making healthy choices by consuming legumes and vegetables but needs to improve fruit and nut intake, increase physical activity, and enhance sleep quality.", "Goals": "Increase weekly fruit and nut consumption, increase exercise sessions, and aim for a consistent 7 hours of sleep per night."}}},
{"input": {"demographics": {"gender": "Male", "age": 35, "height_cm": 180, "weight_kg": 75}, "physical_activity": {"weekly_steps": 8500, "zone_minutes": 120, "exercise_sessions": 2}, "sleep": {"average_hours_per_night": 6}, "dietary_preference": {"declared_diet": "Mediterranean", "allergies": "no allergies"}, "weekly_food_consumption": {"dairy_liters": 1, "legumes_grams": 200, "meat_grams": 300, "fruits_grams": 500, "vegetables_grams": 600, "grains_grams": 700, "nuts_grams": 100, "water_liters": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "Your diet follows a Mediterranean style, but there could be improvements in fruit and nut consumption for better overall nutrient intake.", "Areas of Improvements": "Fruits are consumed in lower quantities. Aim to increase fruit intake from 500g/week to around 800g/week. Additionally, nuts could be increased to about 200g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with honey, mixed berries, and a handful of walnuts.", "Lunch: Quinoa salad with cherry tomatoes, cucumbers, feta cheese, and a generous serving of chickpeas.", "Dinner: Grilled salmon with a side of roasted vegetables and a mixed fruit salad for dessert."]}, "Physical Activity": {"Activity Assessment": "Your physical activity level is moderate, with a decent step count and regular exercise sessions.", "Zone Minutes and Intensity Feedback": "You are meeting the recommended zone minutes for cardio, but consider increasing intensity during exercise sessions.", "Strength/Cardio Tips": "Incorporate strength training twice a week to improve muscle tone and increase metabolism.", "Weekly Goals": "Aim to increase your weekly steps to between 10,000-12,000 and add an additional exercise session this month."}, "Sleep & Well-being": {"Sleep Review": "An average of 6 hours of sleep per night is below the recommended range for optimal health.", "Suggestions": "Try to aim for at least 7-8 hours of sleep. Establish a consistent bedtime routine and limit screen time before bed."}, "Weekly Summary": {"Summary": "Your overall health could benefit from increased fruit and nut intake, along with better sleep quality.", "Goals": "Increase weekly fruit and nut consumption, aim for more step activity, enhance exercise intensity, and improve sleep duration."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height_cm": 178, "Weight_kg": 85}, "Physical Activity": {"Weekly_Steps": 8000, "Zone_Minutes_per_week": 100, "Exercise_Sessions_per_week": 2}, "Sleep": {"Average_hours_per_night": 6.5}, "Dietary_Preference": {"Declared_Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly_Food_Consumption": {"Dairy_liters": 1, "Legumes_grams": 200, "Meat_grams": 400, "Fruits_grams": 300, "Vegetables_grams": 400, "Grains_grams": 600, "Nuts_grams": 50, "Water_liters": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "Your diet reflects a Mediterranean style but has a low intake of fruits and nuts, which are essential for additional vitamins and healthy fats.", "Areas of Improvements": "Increase fruit consumption from 300g/week to around 800g/week and nuts from 50g/week to 150g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with honey, mixed berries, and a handful of walnuts.", "Lunch: Quinoa salad with chickpeas, diced cucumber, bell peppers, cherry tomatoes, topped with olive oil and lemon juice.", "Dinner: Grilled salmon with a side of sautéed spinach and a large mixed greens salad with avocado and pumpkin seeds."]}, "Physical Activity": {"Activity Assessment": "You are meeting the minimum activity recommendations but could benefit from increasing your exercise sessions and intensity.", "Zone Minutes and Intensity Feedback": "100 zone minutes is a good start, but aim for higher intensity workouts to improve cardiovascular health.", "Strength/Cardio Tips": "Incorporate two strength training sessions per week focusing on major muscle groups. Consider adding a brisk walk or jog to increase your cardio time.", "Weekly Goals": "Aim for 150 zone minutes per week and increase exercise sessions to at least 3."}, "Sleep & Well-being": {"Sleep Review": "Currently, you are getting less than the recommended amount of sleep. Quality sleep is crucial for recovery and overall health.", "Suggestions": "Aim to increase your sleep to 7-8 hours per night. Establish a bedtime routine by going to bed and waking up at the same time each day, and limiting screen time before bed."}, "Weekly Summary": {"Summary": "Your current diet and activity levels indicate room for improvement, particularly in fruit and nut consumption, physical activity intensity, and sleep duration.", "Goals": "Increase fruit and nut intake, aim for more physical activity, and improve your sleep hygiene to enhance overall well-being."}}},
{"input": {"demographics": {"gender": "Male", "age": 35, "height_cm": 180, "weight_kg": 85}, "physical_activity": {"weekly_steps": 8000, "zone_minutes_per_week": 120, "exercise_sessions_per_week": 2}, "sleep": {"average_hours_per_night": 6}, "dietary_preference": {"declared_diet": "Mediterranean", "allergies": "none"}, "weekly_food_consumption": {"dairy_liters": 1, "legumes_grams": 300, "meat_grams": 200, "fruits_grams": 200, "vegetables_grams": 400, "grains_grams": 700, "nuts_grams": 50, "water_liters": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The current diet shows a reasonable intake of vegetables and legumes. However, fruit intake is low, and nut consumption could be higher for healthy fats.", "Areas of Improvements": "Increase fruit intake from 200g/week to around 800g/week and nuts from 50g/week to about 150g/week.", "Suggested Meals": ["Breakfast: Greek yogurt with mixed berries and a handful of almonds.", "Lunch: Quinoa salad with chickpeas, tomatoes, cucumbers, and a lemon-olive oil dressing.", "Snack: An apple with a tablespoon of almond butter.", "Dinner: Grilled salmon with a side of roasted vegetables and a small serving of whole-grain couscous."]}, "Physical Activity": {"Activity Assessment": "Current physical activity levels are moderate with a good number of weekly zone minutes.", "Zone Minutes and Intensity Feedback": "120 minutes of moderate activity is beneficial, but consider incorporating higher intensity sessions to boost cardiovascular health.", "Strength/Cardio Tips": "Try to add one more exercise session focusing on strength training per week to improve muscle mass and metabolism.", "Weekly Goals": "Aim for at least 150 zone minutes of activity and include 3 exercise sessions per week."}, "Sleep & Well-being": {"Sleep Review": "Sleeping an average of 6 hours is below the recommended amount for optimal health and recovery.", "Suggestions": "Try establishing a consistent bedtime routine and limit screen time before bed to improve sleep duration and quality."}, "Weekly Summary": {"Summary": "This profile has a balanced dietary approach but needs increased fruit and nut intake while maintaining physical activity levels.", "Goals": "Increase fruit and nut consumption, aim for 3 exercise sessions per week, and improve sleep quality."}}},
{"input": {"Gender": "Male", "Age": 34, "Height (cm)": 178, "Weight (kg)": 85, "Physical Activity": {"Weekly Steps": 7000, "Zone Minutes per week": 100, "Exercise Sessions per week": 2}, "Sleep": {"Average hours per night": 6}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy (liters)": 1, "Legumes (grams)": 400, "Meat (grams)": 600, "Fruits (grams)": 300, "Vegetables (grams)": 400, "Grains (grams)": 500, "Nuts (grams)": 150, "Water (liters)": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "The user follows a Mediterranean diet but has low fruit intake and moderate vegetable consumption. There is a good variety of proteins but an opportunity to enhance fruit and vegetable variety.", "Areas of Improvements": "Increase fruit intake from 300g/week to around 700g/week and consider enhancing vegetable intake to 800g/week.", "Suggested Meals": ["Greek yogurt with mixed berries and a sprinkle of nuts for breakfast.", "A quinoa salad with chickpeas, bell peppers, and olive oil dressing for lunch.", "Roasted vegetables with a grilled chicken breast for dinner."]}, "Physical Activity": {"Activity Assessment": "The user engages in a moderate amount of physical activity with 7000 steps and 100 zone minutes weekly but exercises only 2 times a week.", "Zone Minutes and Intensity Feedback": "Aim to increase zone minutes by incorporating shorter, higher-intensity workouts or additional walking sessions.", "Strength/Cardio Tips": "Include a mix of strength training and cardio in the 2 weekly sessions. Consider adding bodyweight exercises or cycling.", "Weekly Goals": "Target increasing weekly steps to 10,000 and exercise sessions to 3."}, "Sleep & Well-being": {"Sleep Review": "The user averages 6 hours of sleep, which is below the recommended 7-9 hours for optimal health.", "Suggestions": "Establish a consistent bedtime routine and limit screen time before bed to improve sleep quality."}, "Weekly Summary": {"Summary": "The user has a generally healthy diet but needs to increase fruit and vegetable intake significantly while enhancing physical activity levels.", "Goals": "Increase fruit and vegetable consumption, improve exercise frequency, and enhance sleep duration."}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 35, "Height": 180, "Weight": 85}, "Physical Activity": {"Weekly Steps": 8000, "Zone Minutes": 90, "Exercise Sessions": 2}, "Sleep": {"AverageHours": 6.5}, "DietaryPreference": {"DeclaredDiet": "Mediterranean", "Allergies": "none"}, "WeeklyFoodConsumption": {"Dairy": 1.5, "Legumes": 200, "Meat": 300, "Fruits": 400, "Vegetables": 500, "Grains": 1000, "Nuts": 150, "Water": 2}}, "output": {"FoodRecommendation": {"OverallAssessment": "The user has a good foundation in their Mediterranean diet, but there is a need for increased fruit and nut consumption.", "AreasofImprovements": "Fruits should be increased to around 800g/week, and nuts to about 250g/week to enhance nutrient intake.", "SuggestedMeals": ["Breakfast: Greek yogurt with honey, mixed berries, and a sprinkle of almonds.", "Lunch: Quinoa salad with cherry tomatoes, cucumber, chickpeas, and a handful of walnuts.", "Dinner: Grilled salmon with a side of sautéed spinach and roasted sweet potatoes."]}, "PhysicalActivity": {"ActivityAssessment": "The user is moderately active but could benefit from increasing both their step count and exercise frequency.", "ZoneMinutesandIntensityFeedback": "90 zone minutes is decent, but consider increasing this to at least 150 for enhanced cardiovascular benefits.", "Strength/CardioTips": "Incorporate strength training twice a week, focusing on major muscle groups, and aim for brisk walks or cycling on non-exercise days.", "WeeklyGoals": "Aim for at least 10,000 steps daily and increase exercise sessions to 3 per week."}, "Sleep&Well-being": {"SleepReview": "With an average of 6.5 hours of sleep, the user may not be allowing enough time for recovery.", "Suggestions": "Aim for 7-8 hours of quality sleep, consider a wind-down routine before bed, and keep electronics out of the bedroom."}, "WeeklySummary": {"Summary": "The user has a balanced dietary approach consistent with Mediterranean principles but should focus on increasing their fruit and nut intake. Physical activity is moderate but could improve significantly.", "Goals": {"NutritionGoals": "Add more fruits and nuts weekly, aiming for specific targets.", "PhysicalActivityGoals": "Reach a minimum of 10,000 steps daily and include regular strength training sessions."}}}},
{"input": {"Demographics": {"Gender": "Male", "Age": 34, "Height": 178, "Weight": 85}, "Physical Activity": {"Weekly Steps": 6000, "Zone Minutes": 100, "Exercise Sessions": 2}, "Sleep": {"Average Hours": 6.5}, "Dietary Preference": {"Declared Diet": "Mediterranean", "Allergies": "no allergies"}, "Weekly Food Consumption": {"Dairy": 1.5, "Legumes": 200, "Meat": 500, "Fruits": 400, "Vegetables": 600, "Grains": 800, "Nuts": 50, "Water": 2}}, "output": {"Food Recommendation": {"Overall Assessment": "Your diet reflects a Mediterranean approach with a decent intake of vegetables and grains. However, fruit and nut consumption is below optimal levels.", "Areas of Improvements": "Consider increasing fruit intake from 400g/week to around 800g/week and nuts from 50g/week to about 150g/week for better heart health and nutrient diversity.", "Suggested Meals": ["Breakfast: Greek yogurt with mixed berries and a sprinkle of almonds.", "Lunch: Quinoa salad with mixed vegetables, chickpeas, and a handful of walnuts.", "Dinner: Grilled chicken with a side of roasted sweet potatoes and steamed broccoli.", "Snacks: Apple slices with peanut butter or a handful of mixed nuts."]}, "Physical Activity": {"Activity Assessment": "You're currently achieving a moderate level of physical activity, but there is room for improvement.", "Zone Minutes and Intensity Feedback": "With 100 zone minutes, consider increasing this to 150 minutes for better cardiovascular health.", "Strength/Cardio Tips": "Incorporate at least one more exercise session each week focusing on strength training, and aim for more consistent walking or activity throughout the week.", "Weekly Goals": "Aim for 10,000 steps/day and 150 minutes of moderate to vigorous activity weekly."}, "Sleep & Well-being": {"Sleep Review": "With an average of 6.5 hours of sleep per night, you are on the lower end of the recommended range for adults.", "Suggestions": "Try to establish a consistent bedtime routine, limit screen time before bed, and aim for at least 7-8 hours of sleep per night for better recovery and overall health."}, "Weekly Summary": {"Summary": "You have a solid foundation with your Mediterranean diet, but small adjustments in fruit and nut consumption, alongside slightly increased physical activity and improved sleep habits, can greatly enhance your overall health.", "Goals": ["Increase fruit intake to 800g/week.", "Consume more nuts (aim for 150g/week).", "Increase overall physical activity to 150 minutes/week.", "Improve average nightly sleep to 7-8 hours."]}}},

             ]

data_path = "/content/diet_finetune_examples.jsonl"
data = []
# with open(data_path, "r") as f:
#     for line in f:
#         item = json.loads(line)
#         user_msg = next(msg["content"] for msg in item["messages"] if msg["role"] == "user")
#         assistant_msg = next(msg["content"] for msg in item["messages"] if msg["role"] == "assistant")
#         prompt = f"<|user|>\n{user_msg}\n<|assistant|>\n{assistant_msg}"
#         data.append({"text": prompt})

data = [{"text": f"<|user|>\n{ex['input']}\n<|assistant|>\n{ex['output']}"} for ex in new_data]
# Convert to Hugging Face Dataset
dataset = Dataset.from_list(data)
dataset = dataset.train_test_split(test_size=0.1)

# Load tokenizer and base model
model_id = "ContactDoctor/Bio-Medical-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --- Use BitsAndBytesConfig for 4-bit loading ---
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config, # Use quantization_config instead of load_in_8bit
    device_map="auto",
    torch_dtype=torch.bfloat16, # Use bfloat16 for compute dtype
    low_cpu_mem_usage=True,
)

model = prepare_model_for_kbit_training(model)

# Apply LoRA PEFT configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# Tokenization function
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./diet-finetuned-model",
    per_device_train_batch_size=1,  # Keep reduced batch size
    per_device_eval_batch_size=1,   # Keep reduced batch size
    num_train_epochs=3,
    logging_dir="./logs",
    save_total_limit=2,
    learning_rate=2e-4,
    fp16=False, # Set to False when using bfloat16 or 4-bit
    bf16=True, # Enable bfloat16 if supported by GPU
    save_strategy="epoch",
    report_to="none",
    gradient_checkpointing=True, # Keep gradient checkpointing enabled
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Start fine-tuning
trainer.train()

# Save final LoRA adapter
model.save_pretrained("./WellBeing_LLM")
tokenizer.save_pretrained("./WellBeing_LLM")

In [ ]:
adapter_path = "AnjaliNV/WellBeing_LLM"
model.push_to_hub(adapter_path, use_auth_token=True)
tokenizer.push_to_hub(adapter_path, use_auth_token=True)

#test

In [ ]:
from time import sleep
from typing_extensions import final
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain.schema import Document
import gc




# === Load FAISS vector store ===
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.load_local(
    "/content/drive/MyDrive/rag_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

foodstore = FAISS.load_local(
    "/content/drive/MyDrive/food_faiss_index",
    embeddings=embeddings,
    allow_dangerous_deserialization=True
)

# === Load LLaMA 3 model with LoRA adapter ===
base_model_id = "ContactDoctor/Bio-Medical-Llama-3-8B"
adapter_path = "AnjaliNV/WellBeing_LLM"

tokenizer = AutoTokenizer.from_pretrained(base_model_id)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()

# === Define RAG-based response generation ===
def generate_step(prompt, temperature=0.3, max_tokens=256):
    try:
        print(f"Generating response with prompt length: {len(prompt)}")
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(model.device)
        input_len = inputs["input_ids"].shape[1]

        print(f"Input token length: {input_len}")

        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=max_tokens,
                temperature=0.3,
                top_p=0.9,
                top_k=50,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                repetition_penalty=1.05,
                use_cache=False,
            )

        generated_tokens = outputs[0][input_len:]
        response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
        print(f"Generated response length: {len(response)}")
        return response
    except Exception as e:
        print(f"Error in generate_step: {e}")
        return f"Error generating response: {str(e)}"

def cleanup_gpu_memory():
    """Enhanced GPU memory cleanup"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    gc.collect()

def generate_with_cot(user_profile, temperature=0.3, max_tokens=512, k=3):

    search1 = f"""
    Using the given user_profile:
    {user_profile}

    1) Find meal recommendations that match the user's dietary preference from user_profile.
    2) Respect their allergies, and include the recommended quantities per week.
    3) Quantities should be appropriate for their diet type, age, and other profile details.
    """

    # Retrieve relevant docs from vectorstore as context
    docs = vectorstore.similarity_search(search1, k=k)
    context = "\n\n".join(doc.page_content for doc in docs)

#     print("fetched context is ", context)

    prompt1 = f"""
      SYSTEM: You are a qualified well being recomender. Use explicit calculations, clear units, and do NOT invent facts not supported by the provided Context.

      CONTEXT (retrieved documents):
      {context}
      --- end of context ---

      Input:
      {user_profile}
      --- end of profile ---


          """

    final_response  = generate_step(prompt1, temperature, max_tokens)
#     food_response  = generate_step(prompt1, temperature, max_tokens)
#     cleanup_gpu_memory()

#     search2 = f"""
#     Given the following user profile:
#     {user_profile}

#     1) Identify the recommended zone minutes of exercise appropriate for the user's age group.
#     2) Suggest suitable strength training and cardiovascular exercises tailored to their profile.
#       """
#     docs = vectorstore.similarity_search(search2, k=k)
#     context2 = "\n\n".join(doc.page_content for doc in docs)
#     print("Physical context length:", len(context2))
#     prompt2 = f"""
#     SYSTEM: You are a qualified physical exercise recommender.

#     CONTEXT (retrieved documents):
#     {context2}
#     --- end of context ---

#     USER PROFILE:
#     {user_profile}
#     --- end of profile ---


#     TASK:- Using the Context and the user profile above, please provide a structured response for Physical Activity including these four parts:
#     1) Activity Assessment: Briefly evaluate the user’s current physical activity level based on their profile and any available info.
#     2) Zone Minutes and Intensity Feedback: Specify the recommended zone minutes (e.g., moderate, vigorous) suitable for their age group, including guidance on exercise intensity.
#     3) Strength/Cardio Tips: Suggest practical strength training and cardiovascular exercises that fit the user’s profile, preferences, and constraints.
#     4) Weekly Goals: Outline realistic weekly physical activity goals, including duration and frequency, to promote health and fitness.
#      """

#     physical_response  = generate_step(prompt2, temperature, max_tokens)
#     cleanup_gpu_memory()


#     search3 = f"""
#     Given the user profile:
#     {user_profile}

#     1) Determine the user’s current sleep duration and mood status.
#     2) Provide tailored recommendations to improve their sleep quality and mood, considering their age and sex.

#     """
#     docs = vectorstore.similarity_search(search3, k=k)
#     context3 = "\n\n".join(doc.page_content for doc in docs)
#     print("Sleep context length:", len(context3))
#     prompt3 = f"""
#     SYSTEM: You are a qualified well-being recommender.
#     CONTEXT (retrieved documents):
#     {context3}
#     --- end of context ---

#     USER PROFILE:
#     {user_profile}
#     --- end of profile ---

#     TASK: Using the provided context and user profile, deliver a structured response on Sleep and Mood covering these two parts:
#     1) Sleep and Mood Review: Analyze the user’s current sleep patterns and mood status based on the available information.
#     2) Suggestions: Offer actionable recommendations to improve the user’s sleep schedule and mood, if needed.
#     """
#     sleep_response  = generate_step(prompt3, temperature, max_tokens)
#     cleanup_gpu_memory()

#     final_response = f"""
#     Food Recommendation:
#     {food_response}
#     Physical Activity Recommendation:
#     {physical_response}
#     Sleep and Mood Recommendation:
#     {sleep_response}
#     """
#     print("final response is ", final_response)
    return final_response

# === Gradio UI ===
iface = gr.Interface(
    fn=generate_with_cot,
    inputs=[gr.Textbox(lines=10, label="Enter User Profile & Data"),
            gr.Slider(0.1, 1.0, value=0.3, label="Temperature"),
            gr.Slider(64, 1024, step=64, value=512, label="Max Tokens"),
            gr.Slider(1, 10, step=1, value=3, label="Top K Documents")],
    outputs=gr.Textbox(label="Health & Wellness Recommendation"),
    title="Chain-of-Thought RAG Wellness Advisor"
)

iface.launch(share=True, debug=True)

